In [1]:
# process the consequences data from MITRE, and return a dictionary of the information.

con_a = "::SCOPE:Availability:IMPACT:DoS: Crash, Exit, or Restart:IMPACT:DoS: Resource Consumption (CPU):IMPACT:DoS: Resource Consumption (Memory):NOTE:An attacker could provide unexpected values and cause a program crash or excessive consumption of resources, such as memory and CPU.::SCOPE:Confidentiality:IMPACT:Read Memory:IMPACT:Read Files or Directories:NOTE:An attacker could read confidential data if they are able to control resource references.::SCOPE:Integrity:SCOPE:Confidentiality:SCOPE:Availability:IMPACT:Modify Memory:IMPACT:Execute Unauthorized Code or Commands:NOTE:An attacker could use malicious input to modify data or possibly alter control flow in unexpected ways, including arbitrary command execution.::"
con_b = "::SCOPE:Confidentiality:IMPACT:Read Application Data:NOTE:The confidentiality of sensitive data may be compromised by the use of a broken or risky cryptographic algorithm.::SCOPE:Integrity:IMPACT:Modify Application Data:NOTE:The integrity of sensitive data may be compromised by the use of a broken or risky cryptographic algorithm.::SCOPE:Accountability:SCOPE:Non-Repudiation:IMPACT:Hide Activities:NOTE:If the cryptographic algorithm is used to ensure the identity of the source of the data (such as digital signatures), then a broken algorithm will compromise this scheme and the source of the data cannot be proven.::"
con_c = "::SCOPE:Access Control:SCOPE:Confidentiality:IMPACT:Bypass Protection Mechanism:IMPACT:Read Application Data:NOTE:The most common attack performed with cross-site scripting involves the disclosure of information stored in user cookies. Typically, a malicious user will craft a client-side script, which -- when parsed by a web browser -- performs some activity (such as sending all site cookies to a given E-mail address). This script will be loaded and run by each user visiting the web site. Since the site requesting to run the script has access to the cookies in question, the malicious script does also.::SCOPE:Integrity:SCOPE:Confidentiality:SCOPE:Availability:IMPACT:Execute Unauthorized Code or Commands:NOTE:In some circumstances it may be possible to run arbitrary code on a victim's computer when cross-site scripting is combined with other flaws.::SCOPE:Confidentiality:SCOPE:Integrity:SCOPE:Availability:SCOPE:Access Control:IMPACT:Execute Unauthorized Code or Commands:IMPACT:Bypass Protection Mechanism:IMPACT:Read Application Data:NOTE:The consequence of an XSS attack is the same regardless of whether it is stored or reflected. The difference is in how the payload arrives at the server. XSS can cause a variety of problems for the end user that range in severity from an annoyance to complete account compromise. Some cross-site scripting vulnerabilities can be exploited to manipulate or steal cookies, create requests that can be mistaken for those of a valid user, compromise confidential information, or execute malicious code on the end user systems for a variety of nefarious purposes. Other damaging attacks include the disclosure of end user files, installation of Trojan horse programs, redirecting the user to some other page or site, running Active X controls (under Microsoft Internet Explorer) from sites that a user perceives as trustworthy, and modifying presentation of content.::"
con_x = "::SCOPE:Confidentiality:IMPACT:Read Application Data::SCOPE:Integrity:IMPACT:Modify Application Data::"

def process_consequences(con):
    data = {}
    scope_list = con.split('::SCOPE:') # Split by ::SCORE:
    scope_list = [x for x in scope_list if x] # remove blank items ('')
    scope_list = [x.replace('::', '') for x in scope_list] # remove :: from end of items

    for i in scope_list: # loop through scopes  
        sub_data={}
        kw = i.split(':')[0] # first word of sentence ''
        j = i.replace(kw, '') # remove keyword from the item
        
        scope_sub_items = re.split(r'(:SCOPE:|:IMPACT:|:NOTE:)',j) 
        scope_sub_items = [x for x in scope_sub_items if x] # remove blank items ('')
        for item in range(0, len(scope_sub_items), 2): # concat following items 1*2, 3*4, etc.
            new_form = scope_sub_items[item].replace(':', '')+'*'+scope_sub_items[item+1]
            c = new_form.split('*')
            if c[0] not in sub_data:
                sub_data[c[0]] = []
            sub_data[c[0]].append(c[1])
        
        if kw not in data:
            data[kw]=sub_data
    return data

import json, re
consequence_data = process_consequences(con_x)
print(json.dumps(consequence_data, indent=2))

# print(consequence_data.keys())
# print(type(consequence_data['Availability']['IMPACT']))
# x=[]
# for key in consequence_data.keys():
#     x.append(key)
# x

{
  "Confidentiality": {
    "IMPACT": [
      "Read Application Data"
    ]
  },
  "Integrity": {
    "IMPACT": [
      "Modify Application Data"
    ]
  }
}


In [4]:
# process the mitigation data from MITRE, and return a dictionary of the information.

mit_a = "::PHASE:Architecture and Design:STRATEGY:Attack Surface Reduction:DESCRIPTION:Consider using language-theoretic security (LangSec) techniques that characterizes inputs using a formal language and builds recognizers for that language. This effectively requires parsing to be a distinct layer that effectively enforces a boundary between raw input and internal data representations, instead of allowing parser code to be scattered throughout the program, where it could be subject to errors or inconsistencies that create weaknesses. [REF-1109] [REF-1110] [REF-1111]::PHASE:Architecture and Design:STRATEGY:Libraries or Frameworks:DESCRIPTION:Use an input validation framework such as Struts or the OWASP ESAPI Validation API. Note that using a framework does not automatically address all input validation problems; be mindful of weaknesses that could arise from misusing the framework itself (CWE-1173).::PHASE:Architecture and Design Implementation:STRATEGY:Attack Surface Reduction:DESCRIPTION:Understand all the potential areas where untrusted inputs can enter your software: parameters or arguments, cookies, anything read from the network, environment variables, reverse DNS lookups, query results, request headers, URL components, e-mail, files, filenames, databases, and any external systems that provide data to the application. Remember that such inputs may be obtained indirectly through API calls.::PHASE:Implementation:STRATEGY:Input Validation:DESCRIPTION:Assume all input is malicious. Use an accept known good input validation strategy, i.e., use a list of acceptable inputs that strictly conform to specifications. Reject any input that does not strictly conform to specifications, or transform it into something that does. When performing input validation, consider all potentially relevant properties, including length, type of input, the full range of acceptable values, missing or extra inputs, syntax, consistency across related fields, and conformance to business rules. As an example of business rule logic, boat may be syntactically valid because it only contains alphanumeric characters, but it is not valid if the input is only expected to contain colors such as red or blue. Do not rely exclusively on looking for malicious or malformed inputs. This is likely to miss at least one undesirable input, especially if the code's environment changes. This can give attackers enough room to bypass the intended validation. However, denylists can be useful for detecting potential attacks or determining which inputs are so malformed that they should be rejected outright.:EFFECTIVENESS:High::PHASE:Architecture and Design:DESCRIPTION:For any security checks that are performed on the client side, ensure that these checks are duplicated on the server side, in order to avoid CWE-602. Attackers can bypass the client-side checks by modifying values after the checks have been performed, or by changing the client to remove the client-side checks entirely. Then, these modified values would be submitted to the server. Even though client-side checks provide minimal benefits with respect to server-side security, they are still useful. First, they can support intrusion detection. If the server receives input that should have been rejected by the client, then it may be an indication of an attack. Second, client-side error-checking can provide helpful feedback to the user about the expectations for valid input. Third, there may be a reduction in server-side processing time for accidental input errors, although this is typically a small savings.::PHASE:Implementation:DESCRIPTION:When your application combines data from multiple sources, perform the validation after the sources have been combined. The individual data elements may pass the validation step but violate the intended restrictions after they have been combined.::PHASE:Implementation:DESCRIPTION:Be especially careful to validate all input when invoking code that crosses language boundaries, such as from an interpreted language to native code. This could create an unexpected interaction between the language boundaries. Ensure that you are not violating any of the expectations of the language with which you are interfacing. For example, even though Java may not be susceptible to buffer overflows, providing a large argument in a call to native code might trigger an overflow.::PHASE:Implementation:DESCRIPTION:Directly convert your input type into the expected data type, such as using a conversion function that translates a string into a number. After converting to the expected data type, ensure that the input's values fall within the expected range of allowable values and that multi-field consistencies are maintained.::PHASE:Implementation:DESCRIPTION:Inputs should be decoded and canonicalized to the application's current internal representation before being validated (CWE-180, CWE-181). Make sure that your application does not inadvertently decode the same input twice (CWE-174). Such errors could be used to bypass allowlist schemes by introducing dangerous inputs after they have been checked. Use libraries such as the OWASP ESAPI Canonicalization control. Consider performing repeated canonicalization until your input does not change any more. This will avoid double-decoding and similar scenarios, but it might inadvertently modify inputs that are allowed to contain properly-encoded dangerous content.::PHASE:Implementation:DESCRIPTION:When exchanging data between components, ensure that both components are using the same character encoding. Ensure that the proper encoding is applied at each interface. Explicitly set the encoding you are using whenever the protocol allows you to do so.::"
mit_b = "::PHASE:Architecture and Design Operation:DESCRIPTION:Very carefully manage the setting, management, and handling of privileges. Explicitly manage trust zones in the software.::PHASE:Architecture and Design:STRATEGY:Separation of Privilege:DESCRIPTION:Follow the principle of least privilege when assigning access rights to entities in a software system.::PHASE:Architecture and Design:STRATEGY:Separation of Privilege:DESCRIPTION:Consider following the principle of separation of privilege. Require multiple conditions to be met before permitting access to a system resource.::"
mit_c = "::PHASE:Architecture and Design:STRATEGY:Libraries or Frameworks:DESCRIPTION:Use a vetted library or framework that does not allow this weakness to occur or provides constructs that make this weakness easier to avoid. Examples of libraries and frameworks that make it easier to generate properly encoded output include Microsoft's Anti-XSS library, the OWASP ESAPI Encoding module, and Apache Wicket.::PHASE:Implementation Architecture and Design:DESCRIPTION:Understand the context in which your data will be used and the encoding that will be expected. This is especially important when transmitting data between different components, or when generating outputs that can contain multiple encodings at the same time, such as web pages or multi-part mail messages. Study all expected communication protocols and data representations to determine the required encoding strategies. For any data that will be output to another web page, especially any data that was received from external inputs, use the appropriate encoding on all non-alphanumeric characters. Parts of the same output document may require different encodings, which will vary depending on whether the output is in the: HTML body Element attributes (such as src=XYZ) URIs JavaScript sections Cascading Style Sheets and style property etc. Note that HTML Entity Encoding is only appropriate for the HTML body. Consult the XSS Prevention Cheat Sheet [REF-724] for more details on the types of encoding and escaping that are needed.::PHASE:Architecture and Design Implementation:STRATEGY:Attack Surface Reduction:DESCRIPTION:Understand all the potential areas where untrusted inputs can enter your software: parameters or arguments, cookies, anything read from the network, environment variables, reverse DNS lookups, query results, request headers, URL components, e-mail, files, filenames, databases, and any external systems that provide data to the application. Remember that such inputs may be obtained indirectly through API calls.:EFFECTIVENESS:Limited::PHASE:Architecture and Design:DESCRIPTION:For any security checks that are performed on the client side, ensure that these checks are duplicated on the server side, in order to avoid CWE-602. Attackers can bypass the client-side checks by modifying values after the checks have been performed, or by changing the client to remove the client-side checks entirely. Then, these modified values would be submitted to the server.::PHASE:Architecture and Design:STRATEGY:Parameterization:DESCRIPTION:If available, use structured mechanisms that automatically enforce the separation between data and code. These mechanisms may be able to provide the relevant quoting, encoding, and validation automatically, instead of relying on the developer to provide this capability at every point where output is generated.::PHASE:Implementation:STRATEGY:Output Encoding:DESCRIPTION:Use and specify an output encoding that can be handled by the downstream component that is reading the output. Common encodings include ISO-8859-1, UTF-7, and UTF-8. When an encoding is not specified, a downstream component may choose a different encoding, either by assuming a default encoding or automatically inferring which encoding is being used, which can be erroneous. When the encodings are inconsistent, the downstream component might treat some character or byte sequences as special, even if they are not special in the original encoding. Attackers might then be able to exploit this discrepancy and conduct injection attacks; they even might be able to bypass protection mechanisms that assume the original encoding is also being used by the downstream component. The problem of inconsistent output encodings often arises in web pages. If an encoding is not specified in an HTTP header, web browsers often guess about which encoding is being used. This can open up the browser to subtle XSS attacks.::PHASE:Implementation:DESCRIPTION:With Struts, write all data from form beans with the bean's filter attribute set to true.::PHASE:Implementation:STRATEGY:Attack Surface Reduction:DESCRIPTION:To help mitigate XSS attacks against the user's session cookie, set the session cookie to be HttpOnly. In browsers that support the HttpOnly feature (such as more recent versions of Internet Explorer and Firefox), this attribute can prevent the user's session cookie from being accessible to malicious client-side scripts that use document.cookie. This is not a complete solution, since HttpOnly is not supported by all browsers. More importantly, XMLHTTPRequest and other powerful browser technologies provide read access to HTTP headers, including the Set-Cookie header in which the HttpOnly flag is set.:EFFECTIVENESS:Defense in Depth::PHASE:Implementation:STRATEGY:Input Validation:DESCRIPTION:Assume all input is malicious. Use an accept known good input validation strategy, i.e., use a list of acceptable inputs that strictly conform to specifications. Reject any input that does not strictly conform to specifications, or transform it into something that does. When performing input validation, consider all potentially relevant properties, including length, type of input, the full range of acceptable values, missing or extra inputs, syntax, consistency across related fields, and conformance to business rules. As an example of business rule logic, boat may be syntactically valid because it only contains alphanumeric characters, but it is not valid if the input is only expected to contain colors such as red or blue. Do not rely exclusively on looking for malicious or malformed inputs. This is likely to miss at least one undesirable input, especially if the code's environment changes. This can give attackers enough room to bypass the intended validation. However, denylists can be useful for detecting potential attacks or determining which inputs are so malformed that they should be rejected outright. When dynamically constructing web pages, use stringent allowlists that limit the character set based on the expected value of the parameter in the request. All input should be validated and cleansed, not just parameters that the user is supposed to specify, but all data in the request, including hidden fields, cookies, headers, the URL itself, and so forth. A common mistake that leads to continuing XSS vulnerabilities is to validate only fields that are expected to be redisplayed by the site. It is common to see data from the request that is reflected by the application server or the application that the development team did not anticipate. Also, a field that is not currently reflected may be used by a future developer. Therefore, validating ALL parts of the HTTP request is recommended. Note that proper output encoding, escaping, and quoting is the most effective solution for preventing XSS, although input validation may provide some defense-in-depth. This is because it effectively limits what will appear in output. Input validation will not always prevent XSS, especially if you are required to support free-form text fields that could contain arbitrary characters. For example, in a chat application, the heart emoticon (<3) would likely pass the validation step, since it is commonly used. However, it cannot be directly inserted into the web page because it contains the < character, which would need to be escaped or otherwise handled. In this case, stripping the < might reduce the risk of XSS, but it would produce incorrect behavior because the emoticon would not be recorded. This might seem to be a minor inconvenience, but it would be more important in a mathematical forum that wants to represent inequalities. Even if you make a mistake in your validation (such as forgetting one out of 100 input fields), appropriate encoding is still likely to protect you from injection-based attacks. As long as it is not done in isolation, input validation is still a useful technique, since it may significantly reduce your attack surface, allow you to detect some attacks, and provide other security benefits that proper encoding does not address. Ensure that you perform input validation at well-defined interfaces within the application. This will help protect the application even if a component is reused or moved elsewhere.::PHASE:Architecture and Design:STRATEGY:Enforcement by Conversion:DESCRIPTION:When the set of acceptable objects, such as filenames or URLs, is limited or known, create a mapping from a set of fixed input values (such as numeric IDs) to the actual filenames or URLs, and reject all other inputs.::PHASE:Operation:STRATEGY:Firewall:DESCRIPTION:Use an application firewall that can detect attacks against this weakness. It can be beneficial in cases in which the code cannot be fixed (because it is controlled by a third party), as an emergency prevention measure while more comprehensive software assurance measures are applied, or to provide defense in depth.:EFFECTIVENESS:Moderate::PHASE:Operation Implementation:STRATEGY:Environment Hardening:DESCRIPTION:When using PHP, configure the application so that it does not use register_globals. During implementation, develop the application so that it does not rely on this feature, but be wary of implementing a register_globals emulation that is subject to weaknesses such as CWE-95, CWE-621, and similar issues.::"
mit_x = "::PHASE:System Configuration:DESCRIPTION:The application configuration should ensure that SSL or an encryption mechanism of equivalent strength and vetted reputation is used for all access-controlled pages.::"

def process_mitigations(mit):
    data = {}
    scope_list = mit.split('::PHASE:') # Split by ::SCORE:
    scope_list = [x for x in scope_list if x] # remove blank items ('')
    scope_list = [x.replace('::', '') for x in scope_list] # remove :: from end of items

    for i in scope_list: # loop through scopes  
        sub_data={}
        kw = i.split(':')[0] # first word of sentence ''
        j = i.replace(kw, '') # remove keyword from the item
        
        scope_sub_items = re.split(r'(:STRATEGY:|:DESCRIPTION:|:EFFECTIVENESS:)',j) 
        scope_sub_items = [x for x in scope_sub_items if x] # remove blank items ('')
        for item in range(0, len(scope_sub_items), 2): # concat following items 1*2, 3*4, etc.
            new_form = scope_sub_items[item].replace(':', '')+'*'+scope_sub_items[item+1]
            c = new_form.split('*')
            if c[0] not in sub_data:
                sub_data[c[0]] = []

            sub_data[c[0]].append(c[1])

        if kw not in data: # if key not present in data
            data[kw]=[]
            data[kw].append(sub_data)

        else: # if key present
            for k, v in sub_data.items(): # add under same keys
                if k in data[kw][0]: # if key found
                    if v not in data[kw][0][k]: # if value not found, add it under value list
                        data[kw][0][k].extend(v)
                else: 
                    data[kw][0][k] = v

    d = {k:v[0] for k,v in data.items()} # structure fix, list to dict.
    return d

import re, json, itertools
mitigation_data = process_mitigations(mit_x)
print(json.dumps(mitigation_data, indent=2))

# print('\nAvailable Keys:', mitigation_data.keys()) 
# print('2nd Key:', list(mitigation_data.keys())[1])
# print(type(mitigation_data['Architecture and Design Operation']))

{
  "System Configuration": {
    "DESCRIPTION": [
      "The application configuration should ensure that SSL or an encryption mechanism of equivalent strength and vetted reputation is used for all access-controlled pages."
    ]
  }
}


In [5]:
# Lookup consequences and mitigation of a given CVE

import couchdb2, json, sys
db_user, db_pass, db_host, db_port, db_name = 'admin', 'Samsung_1234', 'localhost', '5984', 'cyvia_dataset'
server = couchdb2.Server("http://%s:%s@%s:%s" % (db_user, db_pass, db_host, db_port))

if server.up(): # if server is up and ready
    print('** Server ('+str(server.version)+') up and running!\n')
    if db_name in server: # already existing database # if db.exists():
        db = server[db_name]
    else: # create database if does not exist
        print('Database '+db_name+' does not exist. Creating database...')
        server.create(db_name)
else: # exit if server not running
    print('Server not responding, exiting program!')
    sys.exit()
    
# CVE_IDs CVE-1999-0001,CVE-1999-0003,CVE-2020-0002,CVE-1999-1136,CVE-2000-0030,CVE-2010-0001,CVE-2015-0001,CVE-2015-0030
lookupitem = 'CVE-2015-0030' # CVE-1999-001 

scope_lookup, note_lookup, strategy_lookup, desc_lookup = 'IMPACT', 'NOTE', 'STRATEGY', 'DESCRIPTION'
print('Finding information for', '\033[1m'+lookupitem+'\033[0m', '...\n')
if lookupitem in db:
    # print(json.dumps(db[lookupitem ]['cwe_consequences'], indent=2))
    # print(db[lookupitem ]['cwe_consequences'].keys())
    
    # Consequences Scope, Impact and notes.
    if db[lookupitem]['cwe_consequences'] == {}:
        print(' * No consequences recorded for', lookupitem)
    else:
        print('\033[1m'+' * Scope:'+'\033[0m', ', '.join(db[lookupitem]['cwe_consequences']))
        # get all impact values from dictionary
        item_impact = [val[scope_lookup] for key, val in db[lookupitem]['cwe_consequences'].items() if scope_lookup in val]
        item_impact = [item for sublist in item_impact for item in sublist] # make one list, remove sublists
        print('\033[1m'+' * Impact:'+'\033[0m', item_impact)
        # Notes
        item_notes = [val[note_lookup] for key, val in db[lookupitem]['cwe_consequences'].items() if note_lookup in val]
        item_notes = [item for sublist in item_notes for item in sublist] # make one list, remove sublists
        print('\033[1m'+' * Note(s):'+'\033[0m', item_notes) # [:2] show first 2 only
        
    if db[lookupitem]['cwe_mitigations'] == {}:
        print(' * No mitigations recorded for', lookupitem)
    else:
        # Mitigation, prevention, strategies and descriptions.
        print('\033[1m'+'\n * Prevention at:'+'\033[0m', ', '.join(db[lookupitem]['cwe_mitigations']), 'phase(s).')
        item_strategy = [val[strategy_lookup] for key, val in db[lookupitem]['cwe_mitigations'].items() if strategy_lookup in val]
        item_strategy = [item for sublist in item_strategy for item in sublist] # make one list, remove sublists
        print('\033[1m'+' * Strategy:'+'\033[0m', item_strategy)
        # Descriptions
        item_desc = [val[desc_lookup] for key, val in db[lookupitem]['cwe_mitigations'].items() if desc_lookup in val]
        item_desc = [item for sublist in item_desc for item in sublist] # make one list, remove sublists    
        print('\033[1m'+' * Description(s):'+'\033[0m', item_desc) # [:2] show first 2 only
    
else:
    print(' * CVE-ID not found', lookupitem)

** Server (3.1.1) up and running!

Finding information for CVE-2015-0030 ...

 * Scope: Integrity, Confidentiality, Availability
 * Impact: ['Execute Unauthorized Code or Commands', 'Read Files or Directories', 'DoS: Crash, Exit, or Restart']
 * Note(s): ['The attacker may be able to create or overwrite critical files that are used to execute code, such as programs or libraries.', 'The attacker may be able read the contents of unexpected files and expose sensitive data. If the targeted file is used for a security mechanism, then the attacker may be able to bypass that mechanism. For example, by reading a password file, the attacker could conduct brute force password guessing attacks in order to break into an account on the system.', 'The attacker may be able to overwrite, delete, or corrupt unexpected critical files such as programs, libraries, or important data. This may prevent the software from working at all and in the case of a protection mechanisms such as authentication, it has 

In [6]:
# Lookup consequences and mitigation of a given CVE
list

lookupitems = ['CVE-1999', 'CVE-1999-0003', 'CVE-2020-0002', 'CVE-1999-0001', 'CVE-1999-1136', 'CVE-2000-0030', 'CVE-2010-0001', 'CVE-2015-0001','CVE-2015-0030']
# lookupitems = ['CVE-1999-1']

def find_con_mit(lookuplist):
    data = {}
    for item in lookuplist:
        # print(item)
        # print('Finding information for', '\033[1m'+item+'\033[0m', '...\n')
        if item in db:
            data[item] = {}
            # Consequences Scope, Impact and notes.
            if db[item]['cwe_consequences'] == {}:
                data[item]['Consequences']={}
            else:
                x=[]
                for key in db[item ]['cwe_consequences'].keys():
                    x.append(key)
                data[item]['Scope'] = x
                # get all impact values from dictionary
                item_impact = [val[scope_lookup] for key, val in db[item]['cwe_consequences'].items() if scope_lookup in val]
                item_impact = [i for sublist in item_impact for i in sublist] # make one list, remove sublists
                data[item]['Impact'] = item_impact
                # Notes
                item_notes = [val[note_lookup] for key, val in db[item]['cwe_consequences'].items() if note_lookup in val]
                item_notes = [i for sublist in item_notes for i in sublist] # make one list, remove sublists

            if db[item]['cwe_mitigations'] == {}:
                data[item]['Mitigations'] = {}
            else:
                # Mitigation, prevention, strategies and descriptions.
                x=[]
                for key in db[item ]['cwe_mitigations'].keys():
                    x.append(key)
                data[item]['Prevention'] = x
                item_strategy = [val[strategy_lookup] for key, val in db[item]['cwe_mitigations'].items() if strategy_lookup in val]
                item_strategy = [i for sublist in item_strategy for i in sublist] # make one list, remove sublists
                data[item]['Strategy'] = item_strategy
                # Descriptions
                item_desc = [val[desc_lookup] for key, val in db[item ]['cwe_mitigations'].items() if desc_lookup in val]
                item_desc = [i for sublist in item_desc for i in sublist] # make one list, remove sublists    

        else:
            data[item] = "CVE-ID not found"
    return data
        
data = find_con_mit(lookupitems)
print(json.dumps(data, indent=2))

{
  "CVE-1999": "CVE-ID not found",
  "CVE-1999-0003": {
    "Consequences": {},
    "Mitigations": {}
  },
  "CVE-2020-0002": {
    "Scope": [
      "Integrity"
    ],
    "Impact": [
      "Modify Memory",
      "DoS: Crash, Exit, or Restart",
      "Execute Unauthorized Code or Commands"
    ],
    "Prevention": [
      "Requirements",
      "Architecture and Design",
      "Build and Compilation",
      "Implementation",
      "Operation"
    ],
    "Strategy": [
      "Language Selection",
      "Libraries or Frameworks",
      "Compilation or Build Hardening",
      "Environment Hardening",
      "Environment Hardening"
    ]
  },
  "CVE-1999-0001": {
    "Scope": [
      "Availability",
      "Confidentiality",
      "Integrity"
    ],
    "Impact": [
      "DoS: Crash, Exit, or Restart",
      "DoS: Resource Consumption (CPU)",
      "DoS: Resource Consumption (Memory)",
      "Read Memory",
      "Read Files or Directories",
      "Modify Memory",
      "Execute Unauthorized C

In [7]:
print(json.dumps(data['CVE-1999-0001'], indent=2))

{
  "Scope": [
    "Availability",
    "Confidentiality",
    "Integrity"
  ],
  "Impact": [
    "DoS: Crash, Exit, or Restart",
    "DoS: Resource Consumption (CPU)",
    "DoS: Resource Consumption (Memory)",
    "Read Memory",
    "Read Files or Directories",
    "Modify Memory",
    "Execute Unauthorized Code or Commands"
  ],
  "Prevention": [
    "Architecture and Design",
    "Architecture and Design Implementation",
    "Implementation"
  ],
  "Strategy": [
    "Attack Surface Reduction",
    "Libraries or Frameworks",
    "Attack Surface Reduction",
    "Input Validation"
  ]
}


In [9]:
# Lookup consequences and mitigation of a given CVE

import couchdb2, json, sys
db_user, db_pass, db_host, db_port, db_name = 'admin', 'Samsung_1234', 'localhost', '5984', 'cyvia_dataset'
server = couchdb2.Server("http://%s:%s@%s:%s" % (db_user, db_pass, db_host, db_port))

if server.up(): # if server is up and ready
    print('** Server ('+str(server.version)+') up and running!\n')
    if db_name in server: # already existing database # if db.exists():
        db = server[db_name]
    else: # create database if does not exist
        print('Database '+db_name+' does not exist. Creating database...')
        server.create(db_name)
else: # exit if server not running
    print('Server not responding, exiting program!')
    sys.exit()
    
# CVE_IDs CVE-1999-0001,CVE-1999-0003,CVE-2020-0002,CVE-1999-1136,CVE-2000-0030,CVE-2010-0001,CVE-2015-0001,CVE-2015-0030
lookupitem = 'CVE-2020-0002' #'CVE-2015-0030' # CVE-1999-001 

scope_lookup, note_lookup, strategy_lookup, desc_lookup = 'IMPACT', 'NOTE', 'STRATEGY', 'DESCRIPTION'
print('Finding information for', '\033[1m'+lookupitem+'\033[0m', '...\n')
if lookupitem in db:
    # print(json.dumps(db[lookupitem ]['cwe_consequences'], indent=2))
    # print(db[lookupitem ]['cwe_consequences'].keys())
    
    # Consequences Scope, Impact and notes.
    if db[lookupitem]['cwe_consequences'] == {}:
        print(' * No consequences recorded for', lookupitem)
    else:
        print('\033[1m'+' * Scope:'+'\033[0m', ', '.join(db[lookupitem]['cwe_consequences']))
        # get all impact values from dictionary
        item_impact = [val[scope_lookup] for key, val in db[lookupitem]['cwe_consequences'].items() if scope_lookup in val]
        item_impact = [item for sublist in item_impact for item in sublist] # make one list, remove sublists
        print('\033[1m'+' * Impact:'+'\033[0m', item_impact)
        # Notes
        item_notes = [val[note_lookup] for key, val in db[lookupitem]['cwe_consequences'].items() if note_lookup in val]
        item_notes = [item for sublist in item_notes for item in sublist] # make one list, remove sublists
        print('\033[1m'+' * Note(s):'+'\033[0m', item_notes) # [:2] show first 2 only
        
    if db[lookupitem]['cwe_mitigations'] == {}:
        print(' * No mitigations recorded for', lookupitem)
    else:
        # Mitigation, prevention, strategies and descriptions.
        print('\033[1m'+'\n * Prevention at:'+'\033[0m', ', '.join(db[lookupitem]['cwe_mitigations']), 'phase(s).')
        item_strategy = [val[strategy_lookup] for key, val in db[lookupitem]['cwe_mitigations'].items() if strategy_lookup in val]
        item_strategy = [item for sublist in item_strategy for item in sublist] # make one list, remove sublists
        print('\033[1m'+' * Strategy:'+'\033[0m', item_strategy)
        # Descriptions
        item_desc = [val[desc_lookup] for key, val in db[lookupitem]['cwe_mitigations'].items() if desc_lookup in val]
        item_desc = [item for sublist in item_desc for item in sublist] # make one list, remove sublists    
        print('\033[1m'+' * Description(s):'+'\033[0m', item_desc) # [:2] show first 2 only
    
else:
    print(' * CVE-ID not found', lookupitem)

** Server (3.1.1) up and running!

Finding information for CVE-2020-0002 ...

 * Scope: Integrity
 * Impact: ['Modify Memory', 'DoS: Crash, Exit, or Restart', 'Execute Unauthorized Code or Commands']
 * Note(s): []

 * Prevention at: Requirements, Architecture and Design, Build and Compilation, Implementation, Operation phase(s).
 * Strategy: ['Language Selection', 'Libraries or Frameworks', 'Compilation or Build Hardening', 'Environment Hardening', 'Environment Hardening']
 * Description(s): ["Use a language that does not allow this weakness to occur or provides constructs that make this weakness easier to avoid. For example, many languages that perform their own memory management, such as Java and Perl, are not subject to buffer overflows. Other languages, such as Ada and C#, typically provide overflow protection, but the protection can be disabled by the programmer. Be wary that a language's interface to native code may still be subject to overflows, even if the language itself is t

In [ ]:
# Lookup consequences and mitigation of a given CVE
list

lookupitems = ['CVE-1999', 'CVE-1999-0003', 'CVE-2020-0002', 'CVE-1999-0001', 'CVE-1999-1136', 'CVE-2000-0030', 'CVE-2010-0001', 'CVE-2015-0001','CVE-2015-0030']
# lookupitems = ['CVE-1999-1']

def find_con_mit(lookuplist):
    found_con, found_mit = False, False
    data = {}
    for item in lookuplist:
        # print(item)
        # print('Finding information for', '\033[1m'+item+'\033[0m', '...\n')
        if item in db:
            data[item] = {}
            # Consequences Scope, Impact and notes.
            if db[item]['cwe_consequences'] == {}:
                data[item]['Consequences'] = "No consequences information found"
            else:
                found_con = True
                x=[]
                for key in db[item ]['cwe_consequences'].keys():
                    x.append(key)
                data[item]['Scope'] = x
                # get all impact values from dictionary
                item_impact = [val[scope_lookup] for key, val in db[item]['cwe_consequences'].items() if scope_lookup in val]
                item_impact = [i for sublist in item_impact for i in sublist] # make one list, remove sublists
                data[item]['Impact'] = item_impact
                # Notes
                item_notes = [val[note_lookup] for key, val in db[item]['cwe_consequences'].items() if note_lookup in val]
                item_notes = [i for sublist in item_notes for i in sublist] # make one list, remove sublists

            if db[item]['cwe_mitigations'] == {}:
                data[item]['Mitigations'] = "No mitigations information found"
            else:
                found_mit = True
                # Mitigation, prevention, strategies and descriptions.
                x=[]
                for key in db[item ]['cwe_mitigations'].keys():
                    x.append(key)
                data[item]['Prevention'] = x
                item_strategy = [val[strategy_lookup] for key, val in db[item]['cwe_mitigations'].items() if strategy_lookup in val]
                item_strategy = [i for sublist in item_strategy for i in sublist] # make one list, remove sublists
                data[item]['Strategy'] = item_strategy
                # Descriptions
                item_desc = [val[desc_lookup] for key, val in db[item ]['cwe_mitigations'].items() if desc_lookup in val]
                item_desc = [i for sublist in item_desc for i in sublist] # make one list, remove sublists    
        
            if found_con==False and found_mit==False: data[item] = "No consequences or mitigation information found"
        else:
            data[item] = "CVE not found"
    return data
        
data = find_con_mit(lookupitems)
print(json.dumps(data, indent=2))

In [11]:
# Approach 2 - Using CyVIA Data
import couchdb2, json, sys
db_user, db_pass, db_host, db_port, db_name = 'admin', 'Samsung_1234', 'localhost', '5984', 'cyvia_dataset'
server = couchdb2.Server("http://%s:%s@%s:%s" % (db_user, db_pass, db_host, db_port))

if server.up(): # if server is up and ready
    print('** Server ('+str(server.version)+') up and running!\n')
    if db_name in server: # already existing database # if db.exists():
        db = server[db_name]
    else: # create database if does not exist
        print('Database '+db_name+' does not exist. Creating database...')
        server.create(db_name)
else: # exit if server not running
    print('Server not responding, exiting program!')
    sys.exit()

def find_con_mit(lookuplist):
    data = {}
    found_con, found_mit = False, False
    
    # find values for these keys from data
    scope_lookup, note_lookup, strategy_lookup, desc_lookup = 'IMPACT', 'NOTE', 'STRATEGY', 'DESCRIPTION'
    
    for item in lookuplist:
        # print(item)
        # print('Finding information for', '\033[1m'+item+'\033[0m', '...\n')
        if item in db:
            data[item] = {}
            # Consequences Scope, Impact and notes.
            if db[item]['cwe_consequences'] == {}:
                data[item]['Consequences'] = "No consequences information found"
            else:
                found_con = True
                x=[]
                for key in db[item ]['cwe_consequences'].keys():
                    x.append(key)
                data[item]['Target'] = x
                # get all impact values for target
                item_impact = [val[scope_lookup] for key, val in db[item]['cwe_consequences'].items() if scope_lookup in val]
                item_impact = [i for sublist in item_impact for i in sublist] # make one list, remove sublists
                data[item]['Impact'] = item_impact
                # Notes
                item_notes = [val[note_lookup] for key, val in db[item]['cwe_consequences'].items() if note_lookup in val]
                item_notes = [i for sublist in item_notes for i in sublist] # make one list, remove sublists

            if db[item]['cwe_mitigations'] == {}:
                data[item]['Mitigations'] = "No mitigations information found"
            else:
                found_mit = True
                # Mitigation, prevention, strategies and descriptions.
                x=[]
                for key in db[item ]['cwe_mitigations'].keys():
                    x.append(key)
                data[item]['Prevention'] = x
                item_strategy = [val[strategy_lookup] for key, val in db[item]['cwe_mitigations'].items() if strategy_lookup in val]
                item_strategy = [i for sublist in item_strategy for i in sublist] # make one list, remove sublists
                data[item]['Strategy'] = item_strategy
                # Descriptions
                item_desc = [val[desc_lookup] for key, val in db[item ]['cwe_mitigations'].items() if desc_lookup in val]
                item_desc = [i for sublist in item_desc for i in sublist] # make one list, remove sublists    
        
            if found_con==False and found_mit==False: data[item] = "No consequences or mitigation information found"
        else:
            data[item] = "CVE not found"
    return data

lookupitems = ['CVE-1999', 'CVE-1999-0003', 'CVE-2020-0002', 'CVE-1999-0001', 'CVE-1999-1136', 'CVE-2000-0030', 'CVE-2010-0001', 'CVE-2015-0001','CVE-2015-0030']

data = find_con_mit(lookupitems)
print(json.dumps(data, indent=2))
# scope_lookup, note_lookup, strategy_lookup, desc_lookup = 'IMPACT', 'NOTE', 'STRATEGY', 'DESCRIPTION'

** Server (3.1.1) up and running!

{
  "CVE-1999": "CVE not found",
  "CVE-1999-0003": "No consequences or mitigation information found",
  "CVE-2020-0002": {
    "Target": [
      "Integrity"
    ],
    "Impact": [
      "Modify Memory",
      "DoS: Crash, Exit, or Restart",
      "Execute Unauthorized Code or Commands"
    ],
    "Prevention": [
      "Requirements",
      "Architecture and Design",
      "Build and Compilation",
      "Implementation",
      "Operation"
    ],
    "Strategy": [
      "Language Selection",
      "Libraries or Frameworks",
      "Compilation or Build Hardening",
      "Environment Hardening",
      "Environment Hardening"
    ]
  },
  "CVE-1999-0001": {
    "Target": [
      "Availability",
      "Confidentiality",
      "Integrity"
    ],
    "Impact": [
      "DoS: Crash, Exit, or Restart",
      "DoS: Resource Consumption (CPU)",
      "DoS: Resource Consumption (Memory)",
      "Read Memory",
      "Read Files or Directories",
      "Modify Memory

In [12]:
# Combined CyVIA data and MITRE API

import couchdb2, json, sys
from urllib.request import urlopen
import json

db_user, db_pass, db_host, db_port, db_name = 'admin', 'Samsung_1234', 'localhost', '5984', 'cyvia_dataset'
server = couchdb2.Server("http://%s:%s@%s:%s" % (db_user, db_pass, db_host, db_port))

if server.up(): # if server is up and ready
    print('** Server ('+str(server.version)+') up and running!\n')
    if db_name in server: # already existing database # if db.exists():
        db = server[db_name]
    else: # create database if does not exist
        print('Database '+db_name+' does not exist. Creating database...')
        server.create(db_name)
else: # exit if server not running
    print('Server not responding, exiting program!')
    sys.exit()

def find_cve_info(cve_id):
    full_data_mitre, short_data_mitre, cyvia_data = {}, {}, {}
    found_con, found_mit = False, False
    
    cve_details_url = "https://cveawg.mitre.org/api/cve/"+cve_id
    # get MITRE API results on the CVE
    try:
        response = urlopen(cve_details_url)
        full_data_mitre = json.loads(response.read())        
    except:
        full_data_mitre[cve_id], short_data_mitre[cve_id] = "CVE not found", "CVE not found"
        
    # find values for these keys from data
    scope_lookup, note_lookup, strategy_lookup, desc_lookup = 'IMPACT', 'NOTE', 'STRATEGY', 'DESCRIPTION'
    
    # print(cve_id)
    # print('Finding information for', '\033[1m'+cve_id+'\033[0m', '...\n')
    if cve_id in db:
        cyvia_data[cve_id] = {}
        
        # MITRE information
        if cve_id not in short_data_mitre:
            # get product version(s)
            prod_vers, prob_types = [], []
            for v in range(0, len(full_data_mitre['containers']['cna']['affected'][0]['versions'])):
                prod_vers.append(full_data_mitre['containers']['cna']['affected'][0]['versions'][v]['version']) 

            for p in range(0, len(full_data_mitre['containers']['cna']['problemTypes'])):
                for pd in range(0, len(full_data_mitre['containers']['cna']['problemTypes'][p]['descriptions'])):
                    prob_types.append(full_data_mitre['containers']['cna']['problemTypes'][p]['descriptions'][pd]['description'])
            
            cyvia_data[cve_id]['description'] = full_data_mitre['containers']['cna']['descriptions'][0]['value']
            cyvia_data[cve_id]['vendor'] = full_data_mitre['containers']['cna']['affected'][0]['vendor']
            cyvia_data[cve_id]['affected_product'] = full_data_mitre['containers']['cna']['affected'][0]['product']
            cyvia_data[cve_id]['version(s)'] = prod_vers
            cyvia_data[cve_id]['problem_type(s)'] = prob_types
            
            # Clear 'n/a' fields!
            if cyvia_data[cve_id]['vendor'] == 'n/a': del cyvia_data[cve_id]['vendor']
            if cyvia_data[cve_id]['affected_product'] == 'n/a': del cyvia_data[cve_id]['affected_product']
            if cyvia_data[cve_id]['version(s)'] == ['n/a']: del cyvia_data[cve_id]['version(s)']
            if cyvia_data[cve_id]['problem_type(s)'] == ['n/a']: del cyvia_data[cve_id]['problem_type(s)']

        # Consequences Scope, Impact and notes.
        if db[cve_id]['cwe_consequences'] != {}:
            found_con = True
            x=[]
            for key in db[cve_id ]['cwe_consequences'].keys():
                x.append(key)                
            cyvia_data[cve_id]['Target(T)'] = [*set(x)]
            # get all impact values for target
            item_impact = [val[scope_lookup] for key, val in db[cve_id]['cwe_consequences'].items() if scope_lookup in val]
            item_impact = [i for sublist in item_impact for i in sublist] # make one list, remove sublists
            cyvia_data[cve_id]['T_Impact(TI)'] = [*set(item_impact)] # remove duplicates
            # Notes
            item_notes = [val[note_lookup] for key, val in db[cve_id]['cwe_consequences'].items() if note_lookup in val]
            item_notes = [i for sublist in item_notes for i in sublist] # make one list, remove sublists
            # if len(item_notes)!=0: cyvia_data[cve_id]['TI_Notes'] = item_desc # [:2] show first 2 only
        
        # Mitigation, prevention, strategies and descriptions.
        if db[cve_id]['cwe_mitigations'] != {}:
            found_mit = True
            x=[]
            for key in db[cve_id]['cwe_mitigations'].keys():
                x.append(key)
            cyvia_data[cve_id]['Prevent(P)'] = [*set(x)]
            item_strategy = [val[strategy_lookup] for key, val in db[cve_id]['cwe_mitigations'].items() if strategy_lookup in val]
            item_strategy = [i for sublist in item_strategy for i in sublist] # make one list, remove sublists
            if len(item_strategy)!=0: cyvia_data[cve_id]['P_Strategy(PS)'] = [*set(item_strategy)] # remove duplicates [*set(item_strategy)]
            # Descriptions
            item_desc = [val[desc_lookup] for key, val in db[cve_id]['cwe_mitigations'].items() if desc_lookup in val]
            item_desc = [i for sublist in item_desc for i in sublist] # make one list, remove sublists
            if len(item_desc)!=0: cyvia_data[cve_id]['PS_Details'] = item_desc # [:2] show first 2 only
        
        # If nothing found
        if found_con==False and found_mit==False: cyvia_data[cve_id] = "No consequences or mitigation information found"

    else:
        cyvia_data[cve_id] = "CVE not found"

    print(json.dumps(cyvia_data, indent=2))
    # return cyvia_data

# cve_id = 'CVE-2020-0002'
# find_cve_info(cve_id)


lookupitems = ['CVE-1999', 'CVE-1999-0003', 'CVE-2020-0002', 'CVE-2022-0018', 'CVE-2022-0074', 'CVE-2022-0030', 'CVE-2021-0007', 'CVE-2021-0065','CVE-2020-0031', 'CVE-2019-0027', 'CVE-2015-0030']
lookupitems = ['CVE-2022-23594', 'CVE-2018-0580', 'CVE-2005-4151', 'CVE-2022-23586', 'CVE-2021-29614', 'CVE-2022-26488', 'CVE-2022-41934', 'CVE-2022-32151', 'CVE-2022-30284', 'CVE-2021-28861']
for cve_id in lookupitems:
    find_cve_info(cve_id)
    print('---------------------------------------------------------\n')

** Server (3.1.1) up and running!

{
  "CVE-2022-23594": {
    "description": "Tensorflow is an Open Source Machine Learning Framework. The TFG dialect of TensorFlow (MLIR) makes several assumptions about the incoming `GraphDef` before converting it to the MLIR-based dialect. If an attacker changes the `SavedModel` format on disk to invalidate these assumptions and the `GraphDef` is then converted to MLIR-based IR then they can cause a crash in the Python interpreter. Under certain scenarios, heap OOB read/writes are possible. These issues have been discovered via fuzzing and it is possible that more weaknesses exist. We will patch them as they are discovered.",
    "vendor": "tensorflow",
    "affected_product": "tensorflow",
    "version(s)": [
      ">= 2.7.0, < 2.8.0"
    ],
    "problem_type(s)": [
      "CWE-125: Out-of-bounds Read"
    ],
    "Target(T)": [
      "Confidentiality"
    ],
    "T_Impact(TI)": [
      "Read Memory"
    ],
    "Prevent(P)": [
      "Architecture and

{
  "CVE-2022-26488": {
    "description": "In Python before 3.10.3 on Windows, local users can gain privileges because the search path is inadequately secured. The installer may allow a local attacker to add user-writable directories to the system search path. To exploit, an administrator must have installed Python for all users and enabled PATH entries. A non-administrative user can trigger a repair that incorrectly adds user-writable paths into PATH, enabling search-path hijacking of other users and system services. This affects Python (CPython) through 3.7.12, 3.8.x through 3.8.12, 3.9.x through 3.9.10, and 3.10.x through 3.10.2.",
    "Target(T)": [
      "Integrity",
      "Availability",
      "Confidentiality"
    ],
    "T_Impact(TI)": [
      "DoS: Crash, Exit, or Restart",
      "Gain Privileges or Assume Identity",
      "Execute Unauthorized Code or Commands",
      "Read Files or Directories"
    ],
    "Prevent(P)": [
      "Architecture and Design Implementation",
     

{
  "CVE-2021-28861": {
    "description": "Python 3.x through 3.10 has an open redirection vulnerability in lib/http/server.py due to no protection against multiple (/) at the beginning of URI path which may leads to information disclosure. NOTE: this is disputed by a third party because the http.server.html documentation page states \"Warning: http.server is not recommended for production. It only implements basic security checks.\"",
    "Target(T)": [
      "Access Control"
    ],
    "T_Impact(TI)": [
      "Gain Privileges or Assume Identity",
      "Bypass Protection Mechanism"
    ],
    "Prevent(P)": [
      "Operation",
      "Architecture and Design",
      "Architecture and Design Implementation",
      "Implementation"
    ],
    "P_Strategy(PS)": [
      "Attack Surface Reduction",
      "Enforcement by Conversion",
      "Firewall",
      "Input Validation"
    ],
    "PS_Details": [
      "Assume all input is malicious. Use an accept known good input validation strategy

In [13]:
lookup = ['CVE-2018-0580', 'CVE-2005-4151', 'CVE-2022-23594', 'CVE-2022-23586', 'CVE-2021-29614', 'CVE-2022-26488', 'CVE-2022-41934', 'CVE-2022-32151', 'CVE-2022-30284', 'CVE-2021-28861', 'CVE-2019-17514', 'CVE-2019-16935', 'CVE-2010-3492', 'CVE-2022-31116', 'CVE-2016-9015', 'CVE-2014-9365', 'CVE-2010-2480', 'CVE-2022-29216', 'CVE-2022-27237', 'CVE-2022-22984', 'CVE-2022-0571', 'CVE-2022-0238', 'CVE-2022-0197', 'CVE-2022-0196', 'CVE-2022-0157', 'CVE-2021-43854', 'CVE-2020-35766', 'CVE-2020-27619', 'CVE-2018-6461', 'CVE-2018-2753', 'CVE-2018-20534', 'CVE-2017-7262', 'CVE-2017-3311', 'CVE-2016-7914', 'CVE-2016-0490', 'CVE-2016-0489', 'CVE-2016-0487', 'CVE-2016-0486', 'CVE-2016-0485', 'CVE-2016-0484', 'CVE-2016-0482', 'CVE-2016-0481', 'CVE-2016-0480', 'CVE-2014-9090', 'CVE-2013-1229', 'CVE-2012-5650', 'CVE-2010-0212', 'CVE-2010-0211', 'CVE-2009-3729', 'CVE-2009-0131', 'CVE-2008-7144', 'CVE-2008-6903', 'CVE-2008-6536', 'CVE-2008-2020', 'CVE-2008-1837', 'CVE-2008-1412', 'CVE-2008-1387', 'CVE-2008-1372', 'CVE-2007-4593', 'CVE-2006-4762', 'CVE-2006-4761', 'CVE-2006-4760', 'CVE-2006-4711', 'CVE-2006-4710', 'CVE-2006-2446', 'CVE-2006-2444', 'CVE-2006-2298', 'CVE-2006-2240', 'CVE-2006-2078', 'CVE-2006-2077', 'CVE-2006-2076', 'CVE-2006-2075', 'CVE-2006-2074', 'CVE-2006-2073', 'CVE-2006-2072', 'CVE-2006-1646', 'CVE-2006-1268', 'CVE-2006-0900', 'CVE-2006-0718', 'CVE-2006-0717', 'CVE-2006-0647', 'CVE-2006-0645', 'CVE-2006-0468', 'CVE-2006-0453', 'CVE-2006-0452', 'CVE-2006-0451', 'CVE-2005-4570', 'CVE-2005-4566', 'CVE-2005-4565', 'CVE-2005-4564', 'CVE-2005-4465', 'CVE-2005-4321', 'CVE-2005-3985', 'CVE-2005-3915', 'CVE-2005-3768', 'CVE-2005-3733', 'CVE-2005-3732', 'CVE-2005-3674', 'CVE-2005-3673', 'CVE-2005-3672', 'CVE-2005-3671', 'CVE-2005-3670', 'CVE-2005-3669', 'CVE-2005-3668', 'CVE-2005-3667', 'CVE-2005-3666', 'CVE-2004-2758', 'CVE-2004-2629', 'CVE-2004-1457', 'CVE-2004-0222', 'CVE-2004-0221', 'CVE-2004-0220', 'CVE-2004-0219', 'CVE-2004-0218', 'CVE-2004-0184', 'CVE-2004-0183', 'CVE-2004-0097', 'CVE-2004-0056', 'CVE-2004-0054', 'CVE-2003-1115', 'CVE-2003-1114', 'CVE-2003-1113', 'CVE-2003-1112', 'CVE-2003-1111', 'CVE-2003-1110', 'CVE-2003-1109', 'CVE-2003-1108', 'CVE-2003-0819', 'CVE-2003-0633', 'CVE-2003-0565', 'CVE-2003-0564', 'CVE-2002-1360', 'CVE-2002-1359', 'CVE-2002-1358', 'CVE-2002-1357', 'CVE-2002-0013', 'CVE-2002-0012', 'CVE-2001-1321', 'CVE-2001-1320', 'CVE-2001-1319', 'CVE-2001-1318', 'CVE-2001-1317', 'CVE-2001-1316', 'CVE-2001-1315', 'CVE-2001-1314', 'CVE-2001-1313', 'CVE-2001-1312', 'CVE-2001-1311', 'CVE-2001-1310', 'CVE-2001-1309', 'CVE-2001-1308', 'CVE-2001-1307', 'CVE-2001-1306', 'CVE-2001-0975', 'CVE-2001-0974', 'CVE-2001-0701', 'CVE-2022-21668', 'CVE-2021-3572', 'CVE-2021-21330', 'CVE-2019-20916', 'CVE-2013-5123', 'CVE-2013-1888', 'CVE-2013-1629', 'CVE-2022-36069', 'CVE-2022-35861', 'CVE-2022-41607', 'CVE-2022-35873', 'CVE-2021-32811', 'CVE-2021-32807', 'CVE-2020-11057', 'CVE-2019-9855', 'CVE-2019-9854', 'CVE-2019-9852', 'CVE-2019-9851', 'CVE-2019-9850', 'CVE-2019-9848', 'CVE-2019-9189', 'CVE-2019-7653', 'CVE-2019-19875', 'CVE-2018-6393', 'CVE-2018-19646', 'CVE-2017-5524', 'CVE-2017-1000483', 'CVE-2015-7489', 'CVE-2014-8165', 'CVE-2014-1933', 'CVE-2012-5379', 'CVE-2011-4355', 'CVE-2008-5102', 'CVE-2006-4980', 'CVE-2022-42919', 'CVE-2022-36087', 'CVE-2022-29217', 'CVE-2022-29205', 'CVE-2022-24761', 'CVE-2022-1941', 'CVE-2021-37678', 'CVE-2020-25596', 'CVE-2020-13948', 'CVE-2016-2166', 'CVE-2016-1853', 'CVE-2015-0692', 'CVE-2015-0290', 'CVE-2010-3081', 'CVE-2009-2940', 'CVE-2008-3294', 'CVE-2008-0553', 'CVE-2007-6515', 'CVE-2007-5137', 'CVE-2007-2877', 'CVE-2004-0150', 'CVE-2001-1375', 'CVE-2022-35918', 'CVE-2022-31177', 'CVE-2021-29239', 'CVE-2021-25216', 'CVE-2016-10561', 'CVE-2013-1665', 'CVE-2013-1664', 'CVE-2011-4213', 'CVE-2011-4212', 'CVE-2011-4211', 'CVE-2011-4137', 'CVE-2011-1364', 'CVE-2010-4255', 'CVE-2010-4157', 'CVE-2010-3858', 'CVE-2005-2483', 'CVE-2022-45061', 'CVE-2022-44900', 'CVE-2022-44561', 'CVE-2022-44560', 'CVE-2022-44054', 'CVE-2022-44053', 'CVE-2022-44052', 'CVE-2022-44051', 'CVE-2022-44050', 'CVE-2022-44049', 'CVE-2022-44048', 'CVE-2022-43754', 'CVE-2022-43753', 'CVE-2022-43306', 'CVE-2022-43305', 'CVE-2022-43304', 'CVE-2022-43303', 'CVE-2022-42969', 'CVE-2022-42965', 'CVE-2022-42044', 'CVE-2022-42043', 'CVE-2022-42042', 'CVE-2022-42041', 'CVE-2022-42040', 'CVE-2022-42039', 'CVE-2022-42038', 'CVE-2022-42037', 'CVE-2022-42036', 'CVE-2022-41945', 'CVE-2022-41931', 'CVE-2022-41909', 'CVE-2022-41908', 'CVE-2022-41887', 'CVE-2022-41387', 'CVE-2022-41386', 'CVE-2022-41385', 'CVE-2022-41384', 'CVE-2022-41383', 'CVE-2022-41382', 'CVE-2022-41381', 'CVE-2022-41380', 'CVE-2022-40812', 'CVE-2022-40811', 'CVE-2022-40810', 'CVE-2022-40809', 'CVE-2022-40808', 'CVE-2022-40807', 'CVE-2022-40806', 'CVE-2022-40805', 'CVE-2022-40432', 'CVE-2022-40431', 'CVE-2022-40430', 'CVE-2022-40429', 'CVE-2022-40428', 'CVE-2022-40427', 'CVE-2022-40426', 'CVE-2022-40425', 'CVE-2022-40424', 'CVE-2022-39280', 'CVE-2022-39254', 'CVE-2022-39227', 'CVE-2022-38887', 'CVE-2022-38886', 'CVE-2022-38885', 'CVE-2022-38884', 'CVE-2022-38883', 'CVE-2022-38882', 'CVE-2022-38881', 'CVE-2022-38880', 'CVE-2022-36834', 'CVE-2022-36126', 'CVE-2022-36100', 'CVE-2022-36099', 'CVE-2022-36070', 'CVE-2022-36065', 'CVE-2022-36040', 'CVE-2022-36027', 'CVE-2022-36024', 'CVE-2022-35920', 'CVE-2022-35871', 'CVE-2022-33977', 'CVE-2022-33684', 'CVE-2022-32552', 'CVE-2022-31575', 'CVE-2022-31571', 'CVE-2022-31518', 'CVE-2022-31516', 'CVE-2022-31471', 'CVE-2022-31255', 'CVE-2022-31124', 'CVE-2022-31117', 'CVE-2022-31015', 'CVE-2022-30885', 'CVE-2022-30877', 'CVE-2022-30688', 'CVE-2022-30298', 'CVE-2022-30286', 'CVE-2022-30034', 'CVE-2022-2996', 'CVE-2022-29202', 'CVE-2022-28802', 'CVE-2022-28696', 'CVE-2022-27896', 'CVE-2022-27271', 'CVE-2022-27177', 'CVE-2022-2634', 'CVE-2022-26236', 'CVE-2022-24859', 'CVE-2022-24801', 'CVE-2022-24065', 'CVE-2022-23651', 'CVE-2022-23599', 'CVE-2022-23472', 'CVE-2022-22846', 'CVE-2022-22817', 'CVE-2022-21866', 'CVE-2022-21817', 'CVE-2022-21728', 'CVE-2022-21716', 'CVE-2022-21712', 'CVE-2022-21699', 'CVE-2022-20419', 'CVE-2022-0718', 'CVE-2022-0391', 'CVE-2021-46823', 'CVE-2021-45082', 'CVE-2021-44255', 'CVE-2021-43837', 'CVE-2021-43818', 'CVE-2021-43572', 'CVE-2021-42771', 'CVE-2021-42576', 'CVE-2021-42561', 'CVE-2021-42343', 'CVE-2021-4189', 'CVE-2021-41848', 'CVE-2021-41281', 'CVE-2021-41250', 'CVE-2021-41213', 'CVE-2021-41168', 'CVE-2021-41131', 'CVE-2021-41125', 'CVE-2021-40839', 'CVE-2021-40831', 'CVE-2021-40830', 'CVE-2021-40829', 'CVE-2021-40828', 'CVE-2021-4007', 'CVE-2021-39317', 'CVE-2021-39271', 'CVE-2021-39182', 'CVE-2021-39158', 'CVE-2021-38305', 'CVE-2021-38112', 'CVE-2021-3737', 'CVE-2021-3733', 'CVE-2021-36829', 'CVE-2021-36667', 'CVE-2021-36359', 'CVE-2021-34552', 'CVE-2021-34363', 'CVE-2021-3426', 'CVE-2021-33880', 'CVE-2021-33571', 'CVE-2021-33509', 'CVE-2021-33430', 'CVE-2021-33026', 'CVE-2021-32839', 'CVE-2021-32831', 'CVE-2021-32677', 'CVE-2021-32674', 'CVE-2021-32633', 'CVE-2021-32618', 'CVE-2021-32052', 'CVE-2021-31997', 'CVE-2021-3177', 'CVE-2021-29921', 'CVE-2021-29572', 'CVE-2021-29571', 'CVE-2021-29567', 'CVE-2021-29548', 'CVE-2021-29539', 'CVE-2021-29514', 'CVE-2021-29513', 'CVE-2021-29512', 'CVE-2021-29510', 'CVE-2021-29471', 'CVE-2021-29421', 'CVE-2021-28957', 'CVE-2021-28667', 'CVE-2021-28363', 'CVE-2021-28359', 'CVE-2021-26605', 'CVE-2021-26551', 'CVE-2021-25322', 'CVE-2021-23336', 'CVE-2021-21419', 'CVE-2021-21394', 'CVE-2021-21393', 'CVE-2021-21392', 'CVE-2021-21333', 'CVE-2021-21332', 'CVE-2021-21317', 'CVE-2021-21274', 'CVE-2021-21273', 'CVE-2021-21241', 'CVE-2021-21240', 'CVE-2021-21239', 'CVE-2021-21238', 'CVE-2021-21236', 'CVE-2021-1585', 'CVE-2021-0979', 'CVE-2020-9306', 'CVE-2020-8950', 'CVE-2020-8897', 'CVE-2020-8492', 'CVE-2020-8315', 'CVE-2020-7645', 'CVE-2020-7212', 'CVE-2020-5741', 'CVE-2020-5740', 'CVE-2020-5252', 'CVE-2020-5227', 'CVE-2020-5215', 'CVE-2020-36242', 'CVE-2020-35678', 'CVE-2020-29651', 'CVE-2020-29396', 'CVE-2020-28402', 'CVE-2020-27783', 'CVE-2020-27589', 'CVE-2020-27384', 'CVE-2020-27351', 'CVE-2020-26943', 'CVE-2020-26268', 'CVE-2020-26263', 'CVE-2020-26244', 'CVE-2020-26222', 'CVE-2020-26116', 'CVE-2020-25659', 'CVE-2020-25658', 'CVE-2020-25489', 'CVE-2020-24715', 'CVE-2020-24584', 'CVE-2020-24583', 'CVE-2020-1747', 'CVE-2020-16977', 'CVE-2020-15801', 'CVE-2020-15720', 'CVE-2020-15523', 'CVE-2020-15415', 'CVE-2020-15348', 'CVE-2020-15271', 'CVE-2020-15251', 'CVE-2020-15207', 'CVE-2020-15193', 'CVE-2020-15163', 'CVE-2020-15142', 'CVE-2020-15141', 'CVE-2020-15101', 'CVE-2020-14422', 'CVE-2020-14343', 'CVE-2020-13757', 'CVE-2020-13388', 'CVE-2020-13258', 'CVE-2020-13144', 'CVE-2020-13124', 'CVE-2020-12878', 'CVE-2020-12005', 'CVE-2020-12003', 'CVE-2020-12001', 'CVE-2020-11999', 'CVE-2020-1192', 'CVE-2020-11888', 'CVE-2020-1171', 'CVE-2020-11073', 'CVE-2020-10799', 'CVE-2020-10735', 'CVE-2020-10289', 'CVE-2019-9948', 'CVE-2019-9947', 'CVE-2019-9740', 'CVE-2019-9674', 'CVE-2019-9636', 'CVE-2019-7617', 'CVE-2019-7537', 'CVE-2019-7411', 'CVE-2019-6690', 'CVE-2019-6446', 'CVE-2019-5729', 'CVE-2019-5629', 'CVE-2019-5010', 'CVE-2019-4000', 'CVE-2019-3575', 'CVE-2019-3558', 'CVE-2019-2435', 'CVE-2019-20907', 'CVE-2019-20807', 'CVE-2019-19911', 'CVE-2019-19588', 'CVE-2019-19275', 'CVE-2019-19274', 'CVE-2019-18874', 'CVE-2019-18355', 'CVE-2019-18348', 'CVE-2019-17664', 'CVE-2019-17626', 'CVE-2019-17526', 'CVE-2019-1727', 'CVE-2019-17019', 'CVE-2019-16729', 'CVE-2019-16056', 'CVE-2019-15903', 'CVE-2019-15796', 'CVE-2019-15795', 'CVE-2019-15790', 'CVE-2019-14859', 'CVE-2019-14853', 'CVE-2019-14347', 'CVE-2019-13611', 'CVE-2019-13597', 'CVE-2019-13404', 'CVE-2019-12805', 'CVE-2019-12761', 'CVE-2019-12576', 'CVE-2019-12573', 'CVE-2019-12410', 'CVE-2019-12408', 'CVE-2019-12087', 'CVE-2019-11507', 'CVE-2019-11340', 'CVE-2019-11324', 'CVE-2019-11236', 'CVE-2019-10633', 'CVE-2019-10160', 'CVE-2019-10138', 'CVE-2019-10099', 'CVE-2018-7889', 'CVE-2018-7756', 'CVE-2018-6353', 'CVE-2018-6012', 'CVE-2018-5773', 'CVE-2018-3650', 'CVE-2018-20852', 'CVE-2018-20406', 'CVE-2018-20325', 'CVE-2018-20061', 'CVE-2018-18256', 'CVE-2018-18254', 'CVE-2018-18074', 'CVE-2018-17707', 'CVE-2018-17175', 'CVE-2018-16858', 'CVE-2018-16168', 'CVE-2018-15747', 'CVE-2018-14649', 'CVE-2018-14647', 'CVE-2018-14572', 'CVE-2018-12175', 'CVE-2018-10903', 'CVE-2018-1061', 'CVE-2018-1060', 'CVE-2018-1000808', 'CVE-2018-1000807', 'CVE-2018-1000802', 'CVE-2018-1000117', 'CVE-2018-1000030', 'CVE-2018-0023', 'CVE-2018-0015', 'CVE-2017-9807', 'CVE-2017-9462', 'CVE-2017-7235', 'CVE-2017-6900', 'CVE-2017-5018', 'CVE-2017-4932', 'CVE-2017-3590', 'CVE-2017-2810', 'CVE-2017-2809', 'CVE-2017-2592', 'CVE-2017-20052', 'CVE-2017-18207', 'CVE-2017-17522', 'CVE-2017-16764', 'CVE-2017-16763', 'CVE-2017-16618', 'CVE-2017-16616', 'CVE-2017-16615', 'CVE-2017-14483', 'CVE-2017-13718', 'CVE-2017-13106', 'CVE-2017-12612', 'CVE-2017-12340', 'CVE-2017-12301', 'CVE-2017-10803', 'CVE-2017-1002150', 'CVE-2017-1000433', 'CVE-2017-1000246', 'CVE-2017-1000158', 'CVE-2017-0906', 'CVE-2016-9950', 'CVE-2016-9949', 'CVE-2016-9910', 'CVE-2016-9909', 'CVE-2016-8742', 'CVE-2016-7036', 'CVE-2016-6716', 'CVE-2016-6581', 'CVE-2016-6580', 'CVE-2016-5851', 'CVE-2016-5699', 'CVE-2016-5636', 'CVE-2016-5598', 'CVE-2016-4972', 'CVE-2016-4560', 'CVE-2016-4043', 'CVE-2016-3180', 'CVE-2016-2542', 'CVE-2016-2533', 'CVE-2016-1580', 'CVE-2016-1494', 'CVE-2016-10651', 'CVE-2016-1000110', 'CVE-2016-0822', 'CVE-2016-0772', 'CVE-2016-0771', 'CVE-2015-7546', 'CVE-2015-6531', 'CVE-2015-5652', 'CVE-2015-5306', 'CVE-2015-5242', 'CVE-2015-5216', 'CVE-2015-5164', 'CVE-2015-5159', 'CVE-2015-4605', 'CVE-2015-4604', 'CVE-2015-4234', 'CVE-2015-4231', 'CVE-2015-3446', 'CVE-2015-3220', 'CVE-2015-3206', 'CVE-2015-2674', 'CVE-2015-2316', 'CVE-2015-20107', 'CVE-2015-1950', 'CVE-2015-1852', 'CVE-2015-1777', 'CVE-2015-1341', 'CVE-2015-1326', 'CVE-2015-0693', 'CVE-2014-8650', 'CVE-2014-7185', 'CVE-2014-7144', 'CVE-2014-7143', 'CVE-2014-6448', 'CVE-2014-6262', 'CVE-2014-5453', 'CVE-2014-5340', 'CVE-2014-4861', 'CVE-2014-4650', 'CVE-2014-4616', 'CVE-2014-3995', 'CVE-2014-3994', 'CVE-2014-3598', 'CVE-2014-3593', 'CVE-2014-3589', 'CVE-2014-3539', 'CVE-2014-3146', 'CVE-2014-3137', 'CVE-2014-3007', 'CVE-2014-2967', 'CVE-2014-2667', 'CVE-2014-2331', 'CVE-2014-1938', 'CVE-2014-1934', 'CVE-2014-1932', 'CVE-2014-1929', 'CVE-2014-1928', 'CVE-2014-1927', 'CVE-2014-1912', 'CVE-2014-1839', 'CVE-2014-1838', 'CVE-2014-1830', 'CVE-2014-1829', 'CVE-2014-1624', 'CVE-2014-1604', 'CVE-2014-0485', 'CVE-2014-0472', 'CVE-2014-0161', 'CVE-2014-0105', 'CVE-2013-7489', 'CVE-2013-7459', 'CVE-2013-7440', 'CVE-2013-7338', 'CVE-2013-7323', 'CVE-2013-7061', 'CVE-2013-7040', 'CVE-2013-6491', 'CVE-2013-6444', 'CVE-2013-6418', 'CVE-2013-6396', 'CVE-2013-5942', 'CVE-2013-5655', 'CVE-2013-5654', 'CVE-2013-5106', 'CVE-2013-5093', 'CVE-2013-4867', 'CVE-2013-4482', 'CVE-2013-4409', 'CVE-2013-4367', 'CVE-2013-4347', 'CVE-2013-4346', 'CVE-2013-4245', 'CVE-2013-4238', 'CVE-2013-4111', 'CVE-2013-2962', 'CVE-2013-2191', 'CVE-2013-2167', 'CVE-2013-2166', 'CVE-2013-2132', 'CVE-2013-2131', 'CVE-2013-2104', 'CVE-2013-2099', 'CVE-2013-2072', 'CVE-2013-2013', 'CVE-2013-1909', 'CVE-2013-1895', 'CVE-2013-1753', 'CVE-2013-1752', 'CVE-2013-1445', 'CVE-2013-1068', 'CVE-2013-1055', 'CVE-2013-0294', 'CVE-2012-6495', 'CVE-2012-6081', 'CVE-2012-6080', 'CVE-2012-5825', 'CVE-2012-5822', 'CVE-2012-5659', 'CVE-2012-5578', 'CVE-2012-5577', 'CVE-2012-5563', 'CVE-2012-5506', 'CVE-2012-5499', 'CVE-2012-5495', 'CVE-2012-5494', 'CVE-2012-5493', 'CVE-2012-5488', 'CVE-2012-5487', 'CVE-2012-5485', 'CVE-2012-5474', 'CVE-2012-4571', 'CVE-2012-4520', 'CVE-2012-4406', 'CVE-2012-4245', 'CVE-2012-3533', 'CVE-2012-3458', 'CVE-2012-3444', 'CVE-2012-3443', 'CVE-2012-2921', 'CVE-2012-2417', 'CVE-2012-2374', 'CVE-2012-2146', 'CVE-2012-2135', 'CVE-2012-1575', 'CVE-2012-1502', 'CVE-2012-1176', 'CVE-2012-1150', 'CVE-2012-0878', 'CVE-2012-0861', 'CVE-2012-0860', 'CVE-2012-0845', 'CVE-2012-0215', 'CVE-2011-4954', 'CVE-2011-4944', 'CVE-2011-4940', 'CVE-2011-4783', 'CVE-2011-4642', 'CVE-2011-4617', 'CVE-2011-4407', 'CVE-2011-4357', 'CVE-2011-4104', 'CVE-2011-4103', 'CVE-2011-3587', 'CVE-2011-3378', 'CVE-2011-2520', 'CVE-2011-1521', 'CVE-2011-1444', 'CVE-2011-1158', 'CVE-2011-1157', 'CVE-2011-1156', 'CVE-2011-1015', 'CVE-2011-0725', 'CVE-2010-4450', 'CVE-2010-4340', 'CVE-2010-3964', 'CVE-2010-3495', 'CVE-2010-3494', 'CVE-2010-3493', 'CVE-2010-2477', 'CVE-2010-2235', 'CVE-2010-2089', 'CVE-2010-1666', 'CVE-2010-1634', 'CVE-2010-1450', 'CVE-2010-1449', 'CVE-2010-0668', 'CVE-2010-0395', 'CVE-2009-5065', 'CVE-2009-5042', 'CVE-2009-5010', 'CVE-2009-4924', 'CVE-2009-4134', 'CVE-2009-4081', 'CVE-2009-3894', 'CVE-2009-3850', 'CVE-2009-3724', 'CVE-2009-3720', 'CVE-2009-3695', 'CVE-2009-3578', 'CVE-2009-2205', 'CVE-2009-0668', 'CVE-2009-0367', 'CVE-2009-0318', 'CVE-2009-0317', 'CVE-2009-0316', 'CVE-2009-0315', 'CVE-2009-0314', 'CVE-2009-0218', 'CVE-2008-6954', 'CVE-2008-6549', 'CVE-2008-6547', 'CVE-2008-6539', 'CVE-2008-5987', 'CVE-2008-5986', 'CVE-2008-5985', 'CVE-2008-5984', 'CVE-2008-5983', 'CVE-2008-5104', 'CVE-2008-5103', 'CVE-2008-5031', 'CVE-2008-4864', 'CVE-2008-4863', 'CVE-2008-4394', 'CVE-2008-4126', 'CVE-2008-4108', 'CVE-2008-4099', 'CVE-2008-3949', 'CVE-2008-3144', 'CVE-2008-3143', 'CVE-2008-3142', 'CVE-2008-2316', 'CVE-2008-2315', 'CVE-2008-1965', 'CVE-2008-1887', 'CVE-2008-1886', 'CVE-2008-1885', 'CVE-2008-1721', 'CVE-2008-1679', 'CVE-2008-0982', 'CVE-2008-0981', 'CVE-2008-0980', 'CVE-2008-0299', 'CVE-2007-6531', 'CVE-2007-6015', 'CVE-2007-5741', 'CVE-2007-4965', 'CVE-2007-4559', 'CVE-2007-4308', 'CVE-2007-2052', 'CVE-2007-1657', 'CVE-2007-1359', 'CVE-2007-1253', 'CVE-2006-7228', 'CVE-2006-1542', 'CVE-2006-1095', 'CVE-2006-0151', 'CVE-2006-0052', 'CVE-2005-3302', 'CVE-2005-3291', 'CVE-2005-3008', 'CVE-2005-2966', 'CVE-2005-2875', 'CVE-2005-2491', 'CVE-2005-0852', 'CVE-2005-0089', 'CVE-2005-0088', 'CVE-2004-2680', 'CVE-2004-1491', 'CVE-2004-1050', 'CVE-2004-0096', 'CVE-2003-0973', 'CVE-2002-1119', 'CVE-2002-0185', 'CVE-2002-0131', 'CVE-2021-20135', 'CVE-2020-5765', 'CVE-2019-3962', 'CVE-2019-3923', 'CVE-2017-6543', 'CVE-2017-5179', 'CVE-2016-9260', 'CVE-2016-9259', 'CVE-2016-1000029', 'CVE-2016-1000028', 'CVE-2014-7280', 'CVE-2014-4980', 'CVE-2020-7984', 'CVE-2020-25619', 'CVE-2020-13912', 'CVE-2020-12608', 'CVE-2019-3980']

In [14]:
# Combined CyVIA data and MITRE API

import couchdb2, json, sys
from urllib.request import urlopen
import json

db_user, db_pass, db_host, db_port, db_name = 'admin', 'Samsung_1234', 'localhost', '5984', 'cyvia_dataset'
server = couchdb2.Server("http://%s:%s@%s:%s" % (db_user, db_pass, db_host, db_port))

if server.up(): # if server is up and ready
    print('** Server ('+str(server.version)+') up and running!\n')
    if db_name in server: # already existing database # if db.exists():
        db = server[db_name]
    else: # create database if does not exist
        print('Database '+db_name+' does not exist. Creating database...')
        server.create(db_name)
else: # exit if server not running
    print('Server not responding, exiting program!')
    sys.exit()

def find_cve_info(cve_id):
#     print(cve_id)
    full_data_mitre, short_data_mitre, cyvia_data = {}, {}, {}
    found_con, found_mit, mitre_url = False, False, False
    
    cve_details_url = "https://cveawg.mitre.org/api/cve/"+cve_id
    # get MITRE API results on the CVE
    try:
        response = urlopen(cve_details_url)
        full_data_mitre = json.loads(response.read())  
        mitre_url = True
    except:
        print('MITRE URL failed')
        full_data_mitre[cve_id], short_data_mitre[cve_id] = "CVE not found", "CVE not found"
        pass
        
    # find values for these keys from data
    scope_lookup, note_lookup, strategy_lookup, desc_lookup = 'IMPACT', 'NOTE', 'STRATEGY', 'DESCRIPTION'
    
    # print('Finding information for', '\033[1m'+cve_id+'\033[0m', '...\n')
    if cve_id in db:
#         print('Found CVE in Database')
        cyvia_data[cve_id] = {}
        
        # MITRE information
        if mitre_url: #  and cve_id not in short_data_mitre
#             print(cve_id)
            # get product version(s)
            prod_vers, prob_types = [], []
            try:
                for v in range(0, len(full_data_mitre['containers']['cna']['affected'][0]['versions'])):
                    prod_vers.append(full_data_mitre['containers']['cna']['affected'][0]['versions'][v]['version']) 
            
                for p in range(0, len(full_data_mitre['containers']['cna']['problemTypes'])):
                    for pd in range(0, len(full_data_mitre['containers']['cna']['problemTypes'][p]['descriptions'])):
                        prob_types.append(full_data_mitre['containers']['cna']['problemTypes'][p]['descriptions'][pd]['description'])

#                 print(full_data_mitre['containers']['cna']['descriptions'][0]['value'])
#                 print(full_data_mitre['containers']['cna']['affected'][0]['vendor'])
#                 print(full_data_mitre['containers']['cna']['affected'][0]['product'])
#                 print(prob_types)
#                 print(prod_vers)
            
                cyvia_data[cve_id]['description'] = full_data_mitre['containers']['cna']['descriptions'][0]['value']
                cyvia_data[cve_id]['vendor'] = full_data_mitre['containers']['cna']['affected'][0]['vendor']
                cyvia_data[cve_id]['affected_product'] = full_data_mitre['containers']['cna']['affected'][0]['product']
                cyvia_data[cve_id]['problem_type(s)'] = prob_types                
                cyvia_data[cve_id]['version(s)'] = prod_vers
                        
            except KeyError: 
                print('Missing information at MITRE for CVE', cve_id)
                cyvia_data[cve_id]['description'] = db[cve_id]['description']
                pass                

        # Consequences Scope, Impact and notes.
        if db[cve_id]['cwe_consequences'] != {}:
            found_con = True
            x=[]
            for key in db[cve_id ]['cwe_consequences'].keys():
                x.append(key)                
            cyvia_data[cve_id]['Target(T)'] = [*set(x)]
            # get all impact values for target
            item_impact = [val[scope_lookup] for key, val in db[cve_id]['cwe_consequences'].items() if scope_lookup in val]
            item_impact = [i for sublist in item_impact for i in sublist] # make one list, remove sublists
            cyvia_data[cve_id]['T_Impact(TI)'] = [*set(item_impact)] # remove duplicates
            # Notes
            item_notes = [val[note_lookup] for key, val in db[cve_id]['cwe_consequences'].items() if note_lookup in val]
            item_notes = [i for sublist in item_notes for i in sublist] # make one list, remove sublists
            # if len(item_notes)!=0: cyvia_data[cve_id]['TI_Notes'] = item_desc # [:2] show first 2 only
        
        # Mitigation, prevention, strategies and descriptions.
        if db[cve_id]['cwe_mitigations'] != {}:
            found_mit = True
            x=[]
            for key in db[cve_id]['cwe_mitigations'].keys():
                x.append(key)
            cyvia_data[cve_id]['Prevent(P)'] = [*set(x)]
            item_strategy = [val[strategy_lookup] for key, val in db[cve_id]['cwe_mitigations'].items() if strategy_lookup in val]
            item_strategy = [i for sublist in item_strategy for i in sublist] # make one list, remove sublists
            if len(item_strategy)!=0: cyvia_data[cve_id]['P_Strategy(PS)'] = [*set(item_strategy)] # remove duplicates [*set(item_strategy)]
            # Descriptions
            item_desc = [val[desc_lookup] for key, val in db[cve_id]['cwe_mitigations'].items() if desc_lookup in val]
            item_desc = [i for sublist in item_desc for i in sublist] # make one list, remove sublists
            if len(item_desc)!=0: cyvia_data[cve_id]['PS_Details'] = item_desc # [:2] show first 2 only
        
        # If nothing found
        if found_con==False and found_mit==False: cyvia_data[cve_id] = "No consequences or mitigation information found"

    else:
        print('Missing information in CyVIA dataset for CVE', cve_id)
        cyvia_data[cve_id] = "CVE not found in CyVIA dataset!"

    #print(json.dumps(cyvia_data, indent=2))
    return cyvia_data

def find_mitigation(lookupitems):
    mitigation_data = {}
    no_con_mit_count = 0
    vendors, affected_products, problem_types, targets, impacts, prevents, p_strategies, p_details = [],[],[],[],[],[],[],[]
    for cve_id in lookupitems:
        cyvia_data = find_cve_info(cve_id)
        # print(json.dumps(cyvia_data, indent=2))
        if cyvia_data[cve_id] == "No consequences or mitigation information found": 
            no_con_mit_count+=1
            continue
        
        if 'vendor' in cyvia_data[cve_id]: 
            if (cyvia_data[cve_id]['vendor'] != 'n/a'): 
                vendors.append(cyvia_data[cve_id]['vendor'])
        if 'affected_product' in cyvia_data[cve_id]: 
            if (cyvia_data[cve_id]['affected_product'] != 'n/a'): 
                affected_products.append(cyvia_data[cve_id]['affected_product'])
        if 'problem_type(s)' in cyvia_data[cve_id]: 
            if (cyvia_data[cve_id]['problem_type(s)'] != ['n/a']): 
                problem_types.extend(cyvia_data[cve_id]['problem_type(s)'])

        if 'Target(T)' in cyvia_data[cve_id]: targets.extend(cyvia_data[cve_id]['Target(T)'])
        if 'T_Impact(TI)' in cyvia_data[cve_id]: impacts.extend(cyvia_data[cve_id]['T_Impact(TI)'])
        if 'Prevent(P)' in cyvia_data[cve_id]: prevents.extend(cyvia_data[cve_id]['Prevent(P)'])
        if 'P_Strategy(PS)' in cyvia_data[cve_id]: p_strategies.extend(cyvia_data[cve_id]['P_Strategy(PS)'])
        if 'PS_Details' in cyvia_data[cve_id]: p_details.extend(cyvia_data[cve_id]['PS_Details'])
    
    print('\nVendors:', [*set(vendors)])
    print('\nAffected Products:', [*set(affected_products)])
    print('\nProblem Types:', [*set(problem_types)])
    
    print('\nTargets:', [*set(targets)])
    print('\nImpacts:', [*set(impacts)])
    print('\nPreventions(P):', [*set(prevents)])
    print('\nP_Strategies(PS):', [*set(p_strategies)])
    print('\nPS_Details:', [*set(p_details)])
    print('No con and mit found for', no_con_mit_count, 'CVE(s)')
    return [*set(p_details)]
    

lookupitems = ['CVE-2005-4151', 'CVE-2022-23586', 'CVE-2022-23594', 'CVE-2005-4151', 'CVE-2022-23594', 'CVE-2022-23586', 'CVE-2021-29614', 'CVE-2022-26488', 'CVE-2022-41934', 'CVE-2022-32151', 'CVE-2022-30284', 'CVE-2021-28861']

# find_mitigation(lookupitems)

** Server (3.1.1) up and running!



In [15]:
x = ['CVE-2005-3008', 'CVE-2008-1885', 'CVE-2001-1309', 'CVE-2006-0451', 'CVE-2022-21668', 'CVE-2008-1721', 'CVE-2015-5216', 'CVE-2004-0184', 'CVE-2017-1000433', 'CVE-2013-7061', 'CVE-2022-36099', 'CVE-2019-4000', 'CVE-2021-37678', 'CVE-2018-19646', 'CVE-2022-31571', 'CVE-2020-11057', 'CVE-2017-13106', 'CVE-2020-12608', 'CVE-2019-9740', 'CVE-2012-5578', 'CVE-2021-29539', 'CVE-2011-4104', 'CVE-2022-40426', 'CVE-2022-24065', 'CVE-2007-4593', 'CVE-2021-32618', 'CVE-2013-4346', 'CVE-2012-5499', 'CVE-2019-16935', 'CVE-2022-35873', 'CVE-2021-28861', 'CVE-2014-0105', 'CVE-2011-3378', 'CVE-2007-6015', 'CVE-2007-1359', 'CVE-2022-43304', 'CVE-2022-38882', 'CVE-2022-27177', 'CVE-2006-4980', 'CVE-2022-32151', 'CVE-2004-2629', 'CVE-2022-45061', 'CVE-2001-1311', 'CVE-2021-42771', 'CVE-2011-4355', 'CVE-2010-3081', 'CVE-2022-0391', 'CVE-2013-4409', 'CVE-2022-23594', 'CVE-2019-15795', 'CVE-2022-31518', 'CVE-2022-22817', 'CVE-2004-0218', 'CVE-2014-0472', 'CVE-2010-3494', 'CVE-2018-15747', 'CVE-2021-41848', 'CVE-2019-16056', 'CVE-2019-2435', 'CVE-2010-1449', 'CVE-2016-9259', 'CVE-2016-6580', 'CVE-2019-19274', 'CVE-2022-41381', 'CVE-2022-41380', 'CVE-2011-4211', 'CVE-2020-14343', 'CVE-2019-9636', 'CVE-2008-6547', 'CVE-2020-12003', 'CVE-2019-12408', 'CVE-2022-36027', 'CVE-2004-0220', 'CVE-2019-13597', 'CVE-2010-2089', 'CVE-2022-42039', 'CVE-2019-11236', 'CVE-2010-3493', 'CVE-2022-41383', 'CVE-2021-39182', 'CVE-2019-5629', 'CVE-2021-32633', 'CVE-2016-9910', 'CVE-2013-0294', 'CVE-2022-41931', 'CVE-2022-30298', 'CVE-2018-1000802', 'CVE-2011-1157', 'CVE-2014-7185', 'CVE-2006-4711', 'CVE-2022-42965', 'CVE-2011-4212', 'CVE-2011-4617', 'CVE-2020-25619', 'CVE-2014-3598', 'CVE-2020-12001', 'CVE-2019-11324', 'CVE-2014-1604', 'CVE-2009-3720', 'CVE-2013-4347', 'CVE-2021-41213', 'CVE-2020-5740', 'CVE-2021-33509', 'CVE-2016-10651', 'CVE-2007-5741', 'CVE-2019-7411', 'CVE-2016-0484', 'CVE-2019-11340', 'CVE-2019-9948', 'CVE-2022-44900', 'CVE-2016-4560', 'CVE-2006-0468', 'CVE-2015-1326', 'CVE-2011-1158', 'CVE-2016-7914', 'CVE-2006-1542', 'CVE-2012-3443', 'CVE-2017-12301', 'CVE-2011-1521', 'CVE-2015-1852', 'CVE-2008-6903', 'CVE-2016-1580', 'CVE-2021-36667', 'CVE-2014-1932', 'CVE-2012-5577', 'CVE-2008-5104', 'CVE-2001-1310', 'CVE-2020-10735', 'CVE-2013-1888', 'CVE-2011-4954', 'CVE-2020-5765', 'CVE-2003-1113', 'CVE-2022-21728', 'CVE-2012-5825', 'CVE-2006-2073', 'CVE-2020-26268', 'CVE-2013-5942', 'CVE-2021-28957', 'CVE-2021-32811', 'CVE-2020-27589', 'CVE-2012-1502', 'CVE-2022-29216', 'CVE-2012-0860', 'CVE-2012-2921', 'CVE-2018-1000807', 'CVE-2015-0693', 'CVE-2005-4321', 'CVE-2021-40828', 'CVE-2022-44561', 'CVE-2020-24583', 'CVE-2015-1777', 'CVE-2014-3137', 'CVE-2008-4394', 'CVE-2021-29513', 'CVE-2009-0315', 'CVE-2021-21274', 'CVE-2022-38885', 'CVE-2020-11999', 'CVE-2019-12573', 'CVE-2008-0982', 'CVE-2022-31255', 'CVE-2021-38305', 'CVE-2022-29217', 'CVE-2021-32052', 'CVE-2009-2940', 'CVE-2021-21394', 'CVE-2021-34363', 'CVE-2020-25659', 'CVE-2021-26551', 'CVE-2020-1192', 'CVE-2022-41386', 'CVE-2019-18874', 'CVE-2013-7338', 'CVE-2008-3949', 'CVE-2022-44048', 'CVE-2021-42561', 'CVE-2014-2331', 'CVE-2007-2052', 'CVE-2007-4559', 'CVE-2013-7040', 'CVE-2018-18254', 'CVE-2022-27271', 'CVE-2008-1387', 'CVE-2015-5652', 'CVE-2022-30034', 'CVE-2003-0819', 'CVE-2017-7235', 'CVE-2021-46823', 'CVE-2013-1752', 'CVE-2007-2877', 'CVE-2022-35871', 'CVE-2008-5985', 'CVE-2019-10160', 'CVE-2002-0131', 'CVE-2022-21866', 'CVE-2011-1156', 'CVE-2012-1575', 'CVE-2013-5106', 'CVE-2009-4134', 'CVE-2020-14422', 'CVE-2002-0013', 'CVE-2011-4944', 'CVE-2021-28363', 'CVE-2006-4760', 'CVE-2014-1624', 'CVE-2022-35861', 'CVE-2014-2667', 'CVE-2014-1934', 'CVE-2016-1000110', 'CVE-2019-9189', 'CVE-2010-2477', 'CVE-2018-5773', 'CVE-2022-44049', 'CVE-2019-12087', 'CVE-2019-5010', 'CVE-2003-0564', 'CVE-2020-15142', 'CVE-2008-3142', 'CVE-2019-9947', 'CVE-2006-2075', 'CVE-2002-1119', 'CVE-2002-1359', 'CVE-2015-2674', 'CVE-2007-6515', 'CVE-2020-5252', 'CVE-2017-1000158', 'CVE-2008-1886', 'CVE-2007-5137', 'CVE-2020-11888', 'CVE-2008-4108', 'CVE-2022-41382', 'CVE-2022-31575', 'CVE-2013-2167', 'CVE-2013-1753', 'CVE-2004-2680', 'CVE-2017-17522', 'CVE-2019-17514', 'CVE-2018-14649', 'CVE-2022-31177', 'CVE-2022-38886', 'CVE-2009-0316', 'CVE-2022-40809', 'CVE-2022-31117', 'CVE-2021-3572', 'CVE-2022-0718', 'CVE-2018-20061', 'CVE-2017-1002150', 'CVE-2008-1372', 'CVE-2020-10289', 'CVE-2010-0668', 'CVE-2007-1657', 'CVE-2022-40427', 'CVE-2008-5983', 'CVE-2021-29614', 'CVE-2022-30877', 'CVE-2009-0317', 'CVE-2014-1938', 'CVE-2019-9851', 'CVE-2013-4367', 'CVE-2016-5598', 'CVE-2017-2810', 'CVE-2020-13124', 'CVE-2022-38887', 'CVE-2009-0668', 'CVE-2022-22846', 'CVE-2005-3670', 'CVE-2014-1830', 'CVE-2014-0161', 'CVE-2018-16168', 'CVE-2010-4340', 'CVE-2003-1111', 'CVE-2014-7143', 'CVE-2022-42044', 'CVE-2009-3695', 'CVE-2019-19911', 'CVE-2022-21699', 'CVE-2012-4406', 'CVE-2017-6900', 'CVE-2005-2483', 'CVE-2021-21239', 'CVE-2022-23472', 'CVE-2016-7036', 'CVE-2005-3667', 'CVE-2022-30284', 'CVE-2001-1321', 'CVE-2004-0150', 'CVE-2021-36359', 'CVE-2015-1950', 'CVE-2022-31516', 'CVE-2021-29921', 'CVE-2022-29202', 'CVE-2008-2316', 'CVE-2004-0222', 'CVE-2022-41607', 'CVE-2016-1000029', 'CVE-2014-1929', 'CVE-2012-0215', 'CVE-2001-1318', 'CVE-2006-2444', 'CVE-2008-4126', 'CVE-2021-43572', 'CVE-2003-1108', 'CVE-2020-13912', 'CVE-2018-7756', 'CVE-2006-2076', 'CVE-2021-40830', 'CVE-2017-13718', 'CVE-2017-18207', 'CVE-2011-4103', 'CVE-2022-24801', 'CVE-2018-1000808', 'CVE-2001-1316', 'CVE-2021-29510', 'CVE-2022-43754', 'CVE-2003-1110', 'CVE-2021-1585', 'CVE-2013-6491', 'CVE-2017-5018', 'CVE-2022-40424', 'CVE-2022-39254', 'CVE-2014-1928', 'CVE-2018-0580', 'CVE-2013-7459', 'CVE-2016-10561', 'CVE-2001-1314', 'CVE-2001-1375', 'CVE-2017-12340', 'CVE-2022-21716', 'CVE-2005-4570', 'CVE-2019-7537', 'CVE-2022-44054', 'CVE-2014-1933', 'CVE-2014-8165', 'CVE-2021-21240', 'CVE-2021-40829', 'CVE-2013-1068', 'CVE-2015-3220', 'CVE-2019-3962', 'CVE-2022-40430', 'CVE-2012-5494', 'CVE-2006-1095', 'CVE-2002-1357', 'CVE-2011-4642', 'CVE-2014-7280', 'CVE-2022-28802', 'CVE-2018-14647', 'CVE-2016-8742', 'CVE-2009-4081', 'CVE-2022-40429', 'CVE-2006-2298', 'CVE-2009-0131', 'CVE-2005-3669', 'CVE-2020-28402', 'CVE-2017-14483', 'CVE-2006-0647', 'CVE-2020-26263', 'CVE-2019-20907', 'CVE-2021-33430', 'CVE-2005-2491', 'CVE-2020-29396', 'CVE-2014-5453', 'CVE-2003-0633', 'CVE-2022-41909', 'CVE-2010-4450', 'CVE-2015-0692', 'CVE-2021-21393', 'CVE-2022-42969', 'CVE-2020-13258', 'CVE-2022-33684', 'CVE-2005-3673', 'CVE-2012-5659', 'CVE-2008-2315', 'CVE-2022-43753', 'CVE-2013-1895', 'CVE-2020-13757', 'CVE-2003-0973', 'CVE-2022-0157', 'CVE-2005-4465', 'CVE-2009-0367', 'CVE-2013-1665', 'CVE-2022-40431', 'CVE-2004-0096', 'CVE-2016-0822', 'CVE-2022-42042', 'CVE-2020-8492', 'CVE-2020-24584', 'CVE-2010-0211', 'CVE-2008-6536', 'CVE-2020-27619', 'CVE-2021-32674', 'CVE-2016-6716', 'CVE-2016-9260', 'CVE-2021-4189', 'CVE-2020-5215', 'CVE-2012-4520', 'CVE-2022-36070', 'CVE-2022-40810', 'CVE-2019-11507', 'CVE-2010-1666', 'CVE-2008-0553', 'CVE-2019-12805', 'CVE-2017-16763', 'CVE-2017-3311', 'CVE-2006-0453', 'CVE-2013-7323', 'CVE-2022-22984', 'CVE-2017-5524', 'CVE-2019-10633', 'CVE-2005-0852', 'CVE-2012-5487', 'CVE-2004-1491', 'CVE-2008-5103', 'CVE-2022-29205', 'CVE-2021-44255', 'CVE-2020-25596', 'CVE-2013-5654', 'CVE-2018-0015', 'CVE-2014-6448', 'CVE-2019-3923', 'CVE-2019-9855', 'CVE-2021-33026', 'CVE-2012-2374', 'CVE-2015-3206', 'CVE-2013-1445', 'CVE-2022-40425', 'CVE-2015-2316', 'CVE-2021-32807', 'CVE-2017-9807', 'CVE-2005-3672', 'CVE-2017-3590', 'CVE-2021-32831', 'CVE-2022-42041', 'CVE-2005-0088', 'CVE-2022-40811', 'CVE-2018-16858', 'CVE-2013-5123', 'CVE-2022-0196', 'CVE-2018-12175', 'CVE-2020-15271', 'CVE-2022-41387', 'CVE-2022-40428', 'CVE-2022-31116', 'CVE-2022-36024', 'CVE-2014-1838', 'CVE-2019-17626', 'CVE-2014-0485', 'CVE-2017-10803', 'CVE-2021-41131', 'CVE-2010-1450', 'CVE-2005-3733', 'CVE-2008-6954', 'CVE-2020-11073', 'CVE-2003-1114', 'CVE-2010-1634', 'CVE-2012-0878', 'CVE-2016-3180', 'CVE-2001-1317', 'CVE-2022-31124', 'CVE-2021-21238', 'CVE-2006-2078', 'CVE-2022-40806', 'CVE-2022-24859', 'CVE-2005-3915', 'CVE-2013-6418', 'CVE-2009-3578', 'CVE-2020-5741', 'CVE-2004-0183', 'CVE-2008-1837', 'CVE-2018-14572', 'CVE-2012-2135', 'CVE-2008-7144', 'CVE-2001-0975', 'CVE-2021-29239', 'CVE-2022-28696', 'CVE-2021-41281', 'CVE-2005-3668', 'CVE-2018-20325', 'CVE-2017-16618', 'CVE-2014-3593', 'CVE-2013-7489', 'CVE-2022-41887', 'CVE-2021-21273', 'CVE-2021-40831', 'CVE-2020-26116', 'CVE-2018-6012', 'CVE-2004-1050', 'CVE-2020-15101', 'CVE-2014-7144', 'CVE-2020-15720', 'CVE-2018-10903', 'CVE-2022-31471', 'CVE-2021-31997', 'CVE-2011-4357', 'CVE-2021-21317', 'CVE-2006-0645', 'CVE-2006-0452', 'CVE-2011-1015', 'CVE-2019-13611', 'CVE-2012-2146', 'CVE-2005-3666', 'CVE-2020-7212', 'CVE-2022-1941', 'CVE-2012-0845', 'CVE-2001-1313', 'CVE-2022-33977', 'CVE-2016-5636', 'CVE-2008-2020', 'CVE-2008-0981', 'CVE-2020-35766', 'CVE-2022-41385', 'CVE-2016-1853', 'CVE-2022-0571', 'CVE-2022-42036', 'CVE-2019-17526', 'CVE-2016-6581', 'CVE-2005-2875', 'CVE-2014-4650', 'CVE-2020-26244', 'CVE-2021-41168', 'CVE-2022-23586', 'CVE-2022-38884', 'CVE-2005-3768', 'CVE-2022-24761', 'CVE-2019-3575', 'CVE-2019-15796', 'CVE-2020-15348', 'CVE-2010-3964', 'CVE-2013-4238', 'CVE-2019-16729', 'CVE-2016-1000028', 'CVE-2006-0900', 'CVE-2019-17019', 'CVE-2008-5984', 'CVE-2010-3858', 'CVE-2014-3539', 'CVE-2008-4099', 'CVE-2002-1360', 'CVE-2015-1341', 'CVE-2012-6081', 'CVE-2018-0023', 'CVE-2013-1055', 'CVE-2006-2074', 'CVE-2020-12005', 'CVE-2014-6262', 'CVE-2015-5242', 'CVE-2007-6531', 'CVE-2016-0489', 'CVE-2019-12410', 'CVE-2001-1320', 'CVE-2020-25658', 'CVE-2022-44053', 'CVE-2012-0861', 'CVE-2021-39317', 'CVE-2021-3177', 'CVE-2004-0097', 'CVE-2006-2446', 'CVE-2020-10799', 'CVE-2015-3446', 'CVE-2014-4861', 'CVE-2021-42343', 'CVE-2011-0725', 'CVE-2017-5179', 'CVE-2020-24715', 'CVE-2022-40807', 'CVE-2018-7889', 'CVE-2019-7617', 'CVE-2013-2072', 'CVE-2012-4245', 'CVE-2021-3426', 'CVE-2008-1412', 'CVE-2022-2996', 'CVE-2014-3007', 'CVE-2005-3732', 'CVE-2022-44050', 'CVE-2022-36087', 'CVE-2010-3492', 'CVE-2022-26488', 'CVE-2022-0238', 'CVE-2019-19588', 'CVE-2005-4564', 'CVE-2010-4157', 'CVE-2020-29651', 'CVE-2018-2753', 'CVE-2006-0717', 'CVE-2014-1912', 'CVE-2020-27783', 'CVE-2013-7440', 'CVE-2008-6539', 'CVE-2004-0056', 'CVE-2021-25216', 'CVE-2018-1060', 'CVE-2019-14853', 'CVE-2020-35678', 'CVE-2021-34552', 'CVE-2021-25322', 'CVE-2017-12612', 'CVE-2018-1061', 'CVE-2021-0979', 'CVE-2021-28359', 'CVE-2020-26943', 'CVE-2020-1747', 'CVE-2012-3458', 'CVE-2021-21236', 'CVE-2019-14347', 'CVE-2016-4043', 'CVE-2016-5699', 'CVE-2003-0565', 'CVE-2020-27384', 'CVE-2017-6543', 'CVE-2011-4783', 'CVE-2011-3587', 'CVE-2013-1229', 'CVE-2013-4867', 'CVE-2022-36834', 'CVE-2002-1358', 'CVE-2010-2235', 'CVE-2009-5010', 'CVE-2015-5306', 'CVE-2022-36069', 'CVE-2019-3558', 'CVE-2015-6531', 'CVE-2012-5506', 'CVE-2006-0151', 'CVE-2020-26222', 'CVE-2022-39227', 'CVE-2022-42038', 'CVE-2016-0482', 'CVE-2019-13404', 'CVE-2005-4566', 'CVE-2012-1176', 'CVE-2021-42576', 'CVE-2014-3589', 'CVE-2020-8950', 'CVE-2022-36065', 'CVE-2020-16977', 'CVE-2013-2132', 'CVE-2017-16616', 'CVE-2001-0974', 'CVE-2002-0012', 'CVE-2016-2542', 'CVE-2016-1494', 'CVE-2016-2533', 'CVE-2006-2077', 'CVE-2009-3729', 'CVE-2006-0718', 'CVE-2012-3444', 'CVE-2006-2240', 'CVE-2013-2013', 'CVE-2013-2191', 'CVE-2016-0487', 'CVE-2014-3995', 'CVE-2020-15801', 'CVE-2015-5164', 'CVE-2017-16764', 'CVE-2014-5340', 'CVE-2017-16615', 'CVE-2004-0219', 'CVE-2015-7546', 'CVE-2021-29567', 'CVE-2013-1629', 'CVE-2016-0486', 'CVE-2021-21419', 'CVE-2021-32677', 'CVE-2013-6444', 'CVE-2013-4482', 'CVE-2009-3894', 'CVE-2012-5485', 'CVE-2018-6461', 'CVE-2012-5650', 'CVE-2022-42043', 'CVE-2022-40805', 'CVE-2019-19275', 'CVE-2020-8315', 'CVE-2021-43818', 'CVE-2001-1308', 'CVE-2001-1319', 'CVE-2020-1171', 'CVE-2008-5986', 'CVE-2022-21817', 'CVE-2012-6080', 'CVE-2009-0218', 'CVE-2021-21241', 'CVE-2016-0481', 'CVE-2006-0052', 'CVE-2017-2809', 'CVE-2016-9950', 'CVE-2022-20419', 'CVE-2021-29514', 'CVE-2021-33880', 'CVE-2009-3850', 'CVE-2022-40432', 'CVE-2006-7228', 'CVE-2021-21332', 'CVE-2019-14859', 'CVE-2009-5065', 'CVE-2012-2417', 'CVE-2013-5093', 'CVE-2003-1112', 'CVE-2021-21330', 'CVE-2020-7984', 'CVE-2011-1444', 'CVE-2021-39158', 'CVE-2016-0480', 'CVE-2012-5474', 'CVE-2013-2166', 'CVE-2022-43303', 'CVE-2003-1109', 'CVE-2020-25489', 'CVE-2021-29572', 'CVE-2022-0197', 'CVE-2019-12761', 'CVE-2008-5987', 'CVE-2021-3737', 'CVE-2011-1364', 'CVE-2020-8897', 'CVE-2006-2072', 'CVE-2009-3724', 'CVE-2021-29571', 'CVE-2022-36040', 'CVE-2022-44560', 'CVE-2021-43837', 'CVE-2008-3294', 'CVE-2022-40808', 'CVE-2021-21392', 'CVE-2017-2592', 'CVE-2001-1312', 'CVE-2021-38112', 'CVE-2013-2099', 'CVE-2012-6495', 'CVE-2008-6549', 'CVE-2008-1887', 'CVE-2022-35920', 'CVE-2018-6353', 'CVE-2018-20534', 'CVE-2008-3143', 'CVE-2022-41945', 'CVE-2020-15141', 'CVE-2022-32552', 'CVE-2019-7653', 'CVE-2018-20406', 'CVE-2014-8650', 'CVE-2012-5493', 'CVE-2022-30885', 'CVE-2010-0212', 'CVE-2019-20916', 'CVE-2010-3495', 'CVE-2012-5379', 'CVE-2016-0490', 'CVE-2022-38881', 'CVE-2008-4864', 'CVE-2020-13948', 'CVE-2006-4710', 'CVE-2012-1150', 'CVE-2010-4255', 'CVE-2022-35918', 'CVE-2019-17664', 'CVE-2022-44051', 'CVE-2022-23651', 'CVE-2019-3980', 'CVE-2021-45082', 'CVE-2021-28667', 'CVE-2019-6690', 'CVE-2021-4007', 'CVE-2017-1000246', 'CVE-2004-0054', 'CVE-2015-4234', 'CVE-2011-2520', 'CVE-2021-21333', 'CVE-2022-36126', 'CVE-2018-3650', 'CVE-2015-5159', 'CVE-2022-42037', 'CVE-2016-0771', 'CVE-2012-3533', 'CVE-2007-4308', 'CVE-2020-15163', 'CVE-2018-1000117', 'CVE-2022-23599', 'CVE-2019-9674', 'CVE-2019-12576', 'CVE-2021-20135', 'CVE-2022-26236', 'CVE-2020-36242', 'CVE-2012-5822', 'CVE-2005-3291', 'CVE-2020-15207', 'CVE-2008-4863', 'CVE-2007-1253', 'CVE-2009-5042', 'CVE-2005-0089', 'CVE-2021-3733', 'CVE-2008-1679', 'CVE-2018-1000030', 'CVE-2022-41384', 'CVE-2019-5729', 'CVE-2018-18256', 'CVE-2018-17175', 'CVE-2022-42919', 'CVE-2008-1965', 'CVE-2013-1664', 'CVE-2001-1307', 'CVE-2011-4213', 'CVE-2020-5227', 'CVE-2012-5488', 'CVE-2021-32839', 'CVE-2021-43854', 'CVE-2022-2634', 'CVE-2021-40839', 'CVE-2019-20807', 'CVE-2017-0906', 'CVE-2016-9909', 'CVE-2016-4972', 'CVE-2001-0701', 'CVE-2020-13144', 'CVE-2012-5563', 'CVE-2021-29471', 'CVE-2005-3302', 'CVE-2016-9949', 'CVE-2018-17707', 'CVE-2014-1927', 'CVE-2013-5655', 'CVE-2020-12878', 'CVE-2021-29548', 'CVE-2019-6446', 'CVE-2005-3985', 'CVE-2020-9306', 'CVE-2013-2104', 'CVE-2005-3674', 'CVE-2014-1829', 'CVE-2004-2758', 'CVE-2020-27351', 'CVE-2022-43306', 'CVE-2019-9854', 'CVE-2015-7489', 'CVE-2022-39280', 'CVE-2014-9365', 'CVE-2017-20052', 'CVE-2014-4980', 'CVE-2022-41908', 'CVE-2021-23336', 'CVE-2017-9462', 'CVE-2014-9090', 'CVE-2015-0290', 'CVE-2013-2962', 'CVE-2019-15903', 'CVE-2017-7262', 'CVE-2019-1727', 'CVE-2021-41125', 'CVE-2005-2966', 'CVE-2011-4407', 'CVE-2001-1306', 'CVE-2014-2967', 'CVE-2021-29421', 'CVE-2015-4604', 'CVE-2022-27237', 'CVE-2009-0314', 'CVE-2016-9015', 'CVE-2019-9850', 'CVE-2008-0299', 'CVE-2009-0318', 'CVE-2022-43305', 'CVE-2019-18355', 'CVE-2020-13388', 'CVE-2016-0772', 'CVE-2004-1457', 'CVE-2021-36829', 'CVE-2021-41250', 'CVE-2019-10138', 'CVE-2014-4616', 'CVE-2013-1909', 'CVE-2020-15251', 'CVE-2022-38880', 'CVE-2022-41934', 'CVE-2010-0395', 'CVE-2020-7645', 'CVE-2022-42040', 'CVE-2018-20852', 'CVE-2016-0485', 'CVE-2022-38883', 'CVE-2012-5495', 'CVE-2019-18348', 'CVE-2019-10099', 'CVE-2010-2480', 'CVE-2018-6393', 'CVE-2019-15790', 'CVE-2009-4924', 'CVE-2022-44052', 'CVE-2006-1646', 'CVE-2014-3146', 'CVE-2014-3994', 'CVE-2022-30688', 'CVE-2006-4762', 'CVE-2006-4761', 'CVE-2021-39271', 'CVE-2009-2205', 'CVE-2022-21712', 'CVE-2011-4137', 'CVE-2013-2131', 'CVE-2019-19875', 'CVE-2015-4605', 'CVE-2012-4571', 'CVE-2015-20107', 'CVE-2015-4231', 'CVE-2006-1268', 'CVE-2013-4111', 'CVE-2018-18074', 'CVE-2019-9852', 'CVE-2022-27896', 'CVE-2014-1839', 'CVE-2013-4245', 'CVE-2011-4940', 'CVE-2019-9848', 'CVE-2008-5031', 'CVE-2022-30286', 'CVE-2005-3671', 'CVE-2001-1315', 'CVE-2020-15523', 'CVE-2020-15415', 'CVE-2013-6396', 'CVE-2008-5102', 'CVE-2021-26605', 'CVE-2022-36100', 'CVE-2005-4151', 'CVE-2003-1115', 'CVE-2008-3144', 'CVE-2016-2166', 'CVE-2008-0980', 'CVE-2005-4565', 'CVE-2017-4932', 'CVE-2022-31015', 'CVE-2017-1000483', 'CVE-2004-0221', 'CVE-2022-40812', 'CVE-2020-15193', 'CVE-2016-5851', 'CVE-2002-0185', 'CVE-2021-29512', 'CVE-2021-33571', 'CVE-2007-4965']

In [16]:
# find_mitigation(x[185:190])
# find_mitigation(x[0:200])
# find_mitigation(x[0:190])
p_dets = find_mitigation(x[4:10])
# find_mitigation(['CVE-2021-46823'])


Vendors: ['pypa']

Affected Products: ['pipenv', 'Ipsilon']

Problem Types: ['CWE-791: Incomplete Filtering of Special Elements', 'Cross-Site Scripting', "CWE-77: Improper Neutralization of Special Elements used in a Command ('Command Injection')", 'CWE-20: Improper Input Validation', "CWE-78: Improper Neutralization of Special Elements used in an OS Command ('OS Command Injection')", 'CWE-427: Uncontrolled Search Path Element']

Targets: ['Integrity', 'Other', 'Availability', 'Access Control', 'Confidentiality']

Impacts: ['DoS: Resource Consumption (Memory)', 'Gain Privileges or Assume Identity', 'Read Files or Directories', 'Read Memory', 'Read Application Data', 'Unexpected State', 'DoS: Resource Consumption (CPU)', 'DoS: Crash, Exit, or Restart', 'Modify Memory', 'Execute Unauthorized Code or Commands', 'Bypass Protection Mechanism', 'Quality Degradation']

Preventions(P): ['Implementation Architecture and Design', 'Architecture and Design Implementation', 'Operation', 'Architect

In [17]:
p_dets[0]

'Compartmentalize the system to have safe areas where trust boundaries can be unambiguously drawn. Do not allow sensitive data to go outside of the trust boundary and always be careful when interfacing with a compartment outside of the safe area. Ensure that appropriate compartmentalization is built into the system design, and the compartmentalization allows for and reinforces privilege separation functionality. Architects and designers should rely on the principle of least privilege to decide the appropriate time to use privileges and the time to drop privileges.'

In [18]:
# summarize text
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

def text_summarize(text, per):
    nlp = spacy.load('en_core_web_sm')
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():                            
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
    return summary

p_dets_short = []
for i in range(len(p_dets)):
    x = text_summarize(p_dets[i], 0.25)
    if x != '':
        print()
        print('Text len', len(p_dets[i]), ':', p_dets[i])
        print('Summary len', len(x), ':', x)
        p_dets_short.append(x)
    else: 
        print()
        print('Short text len', len(p_dets[i]), ':', p_dets[i])
        p_dets_short.append(p_dets[i])


Text len 569 : Compartmentalize the system to have safe areas where trust boundaries can be unambiguously drawn. Do not allow sensitive data to go outside of the trust boundary and always be careful when interfacing with a compartment outside of the safe area. Ensure that appropriate compartmentalization is built into the system design, and the compartmentalization allows for and reinforces privilege separation functionality. Architects and designers should rely on the principle of least privilege to decide the appropriate time to use privileges and the time to drop privileges.
Summary len 154 : Architects and designers should rely on the principle of least privilege to decide the appropriate time to use privileges and the time to drop privileges.

Short text len 262 : Use an input validation framework such as Struts or the OWASP ESAPI Validation API. Note that using a framework does not automatically address all input validation problems; be mindful of weaknesses that could arise fro


Text len 606 : To help mitigate XSS attacks against the user's session cookie, set the session cookie to be HttpOnly. In browsers that support the HttpOnly feature (such as more recent versions of Internet Explorer and Firefox), this attribute can prevent the user's session cookie from being accessible to malicious client-side scripts that use document.cookie. This is not a complete solution, since HttpOnly is not supported by all browsers. More importantly, XMLHTTPRequest and other powerful browser technologies provide read access to HTTP headers, including the Set-Cookie header in which the HttpOnly flag is set.
Summary len 244 : In browsers that support the HttpOnly feature (such as more recent versions of Internet Explorer and Firefox), this attribute can prevent the user's session cookie from being accessible to malicious client-side scripts that use document.cookie.

Short text len 412 : Understand all the potential areas where untrusted inputs can enter your software: parameter


Text len 485 : Be especially careful to validate all input when invoking code that crosses language boundaries, such as from an interpreted language to native code. This could create an unexpected interaction between the language boundaries. Ensure that you are not violating any of the expectations of the language with which you are interfacing. For example, even though Java may not be susceptible to buffer overflows, providing a large argument in a call to native code might trigger an overflow.
Summary len 149 : Be especially careful to validate all input when invoking code that crosses language boundaries, such as from an interpreted language to native code.

Short text len 254 : When exchanging data between components, ensure that both components are using the same character encoding. Ensure that the proper encoding is applied at each interface. Explicitly set the encoding you are using whenever the protocol allows you to do so.


In [19]:
p_dets_short

['Architects and designers should rely on the principle of least privilege to decide the appropriate time to use privileges and the time to drop privileges.',
 'Use an input validation framework such as Struts or the OWASP ESAPI Validation API. Note that using a framework does not automatically address all input validation problems; be mindful of weaknesses that could arise from misusing the framework itself (CWE-1173).',
 'Attackers can bypass the client-side checks by modifying values after the checks have been performed, or by changing the client to remove the client-side checks entirely.For any security checks that are performed on the client side, ensure that these checks are duplicated on the server side, in order to avoid CWE-602.',
 'This will avoid double-decoding and similar scenarios, but it might inadvertently modify inputs that are allowed to contain properly-encoded dangerous content.',
 'For any data that will be output to another web page, especially any data that was r

In [20]:
# Final version Mitigation
# Combined CyVIA data and MITRE API

import couchdb2, json, sys
from urllib.request import urlopen
import json
# summarize text
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

db_user, db_pass, db_host, db_port, db_name = 'admin', 'Samsung_1234', 'localhost', '5984', 'cyvia_dataset'
server = couchdb2.Server("http://%s:%s@%s:%s" % (db_user, db_pass, db_host, db_port))

if server.up(): # if server is up and ready
    print('** Server ('+str(server.version)+') up and running!\n')
    if db_name in server: # already existing database # if db.exists():
        db = server[db_name]
    else: # create database if does not exist
        print('Database '+db_name+' does not exist. Creating database...')
        server.create(db_name)
else: # exit if server not running
    print('Server not responding, exiting program!')
    sys.exit()

def text_summarize(text, per):
    nlp = spacy.load('en_core_web_sm')
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():                            
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
    return summary
    
def find_cve_info(cve_id):
    full_data_mitre, short_data_mitre, cyvia_data = {}, {}, {}
    found_con, found_mit, mitre_url = False, False, False
    
    cve_details_url = "https://cveawg.mitre.org/api/cve/"+cve_id
    # get MITRE API results on the CVE
    try:
        response = urlopen(cve_details_url)
        full_data_mitre = json.loads(response.read())  
        mitre_url = True
    except:
        print('MITRE URL failed')
        full_data_mitre[cve_id], short_data_mitre[cve_id] = "CVE not found", "CVE not found"
        pass
        
    # find values for these keys from data
    scope_lookup, note_lookup, strategy_lookup, desc_lookup = 'IMPACT', 'NOTE', 'STRATEGY', 'DESCRIPTION'
    
    if cve_id in db:
        cyvia_data[cve_id] = {}
        
        # MITRE information
        if mitre_url: #  and cve_id not in short_data_mitre
            # get product version(s)
            prod_vers, prob_types = [], []
            try:
                for v in range(0, len(full_data_mitre['containers']['cna']['affected'][0]['versions'])):
                    prod_vers.append(full_data_mitre['containers']['cna']['affected'][0]['versions'][v]['version']) 
            
                for p in range(0, len(full_data_mitre['containers']['cna']['problemTypes'])):
                    for pd in range(0, len(full_data_mitre['containers']['cna']['problemTypes'][p]['descriptions'])):
                        prob_types.append(full_data_mitre['containers']['cna']['problemTypes'][p]['descriptions'][pd]['description'])
            
                cyvia_data[cve_id]['description'] = full_data_mitre['containers']['cna']['descriptions'][0]['value']
                cyvia_data[cve_id]['vendor'] = full_data_mitre['containers']['cna']['affected'][0]['vendor']
                cyvia_data[cve_id]['affected_product'] = full_data_mitre['containers']['cna']['affected'][0]['product']
                cyvia_data[cve_id]['problem_type(s)'] = prob_types                
                cyvia_data[cve_id]['version(s)'] = prod_vers
                        
            except KeyError: 
                print('Missing information at MITRE for CVE', cve_id)
                cyvia_data[cve_id]['description'] = db[cve_id]['description']
                pass                

        # Consequences Scope, Impact and notes.
        if db[cve_id]['cwe_consequences'] != {}:
            found_con = True
            x=[]
            for key in db[cve_id ]['cwe_consequences'].keys():
                x.append(key)                
            cyvia_data[cve_id]['Target(T)'] = [*set(x)]
            # get all impact values for target
            item_impact = [val[scope_lookup] for key, val in db[cve_id]['cwe_consequences'].items() if scope_lookup in val]
            item_impact = [i for sublist in item_impact for i in sublist] # make one list, remove sublists
            cyvia_data[cve_id]['T_Impact(TI)'] = [*set(item_impact)] # remove duplicates
            # Notes
            item_notes = [val[note_lookup] for key, val in db[cve_id]['cwe_consequences'].items() if note_lookup in val]
            item_notes = [i for sublist in item_notes for i in sublist] # make one list, remove sublists
            # if len(item_notes)!=0: cyvia_data[cve_id]['TI_Notes'] = item_desc # [:2] show first 2 only
        
        # Mitigation, prevention, strategies and descriptions.
        if db[cve_id]['cwe_mitigations'] != {}:
            found_mit = True
            x=[]
            for key in db[cve_id]['cwe_mitigations'].keys():
                x.append(key)
            cyvia_data[cve_id]['Prevent(P)'] = [*set(x)]
            item_strategy = [val[strategy_lookup] for key, val in db[cve_id]['cwe_mitigations'].items() if strategy_lookup in val]
            item_strategy = [i for sublist in item_strategy for i in sublist] # make one list, remove sublists
            if len(item_strategy)!=0: cyvia_data[cve_id]['P_Strategy(PS)'] = [*set(item_strategy)] # remove duplicates [*set(item_strategy)]
            # Descriptions
            item_desc = [val[desc_lookup] for key, val in db[cve_id]['cwe_mitigations'].items() if desc_lookup in val]
            item_desc = [i for sublist in item_desc for i in sublist] # make one list, remove sublists
            if len(item_desc)!=0: cyvia_data[cve_id]['PS_Details'] = item_desc # [:2] show first 2 only
        
        # If nothing found
        if found_con==False and found_mit==False: cyvia_data[cve_id] = "No consequences or mitigation information found"

    else:
        print('Missing information in CyVIA dataset for CVE', cve_id)
        cyvia_data[cve_id] = "CVE not found in CyVIA dataset!"

    #print(json.dumps(cyvia_data, indent=2))
    return cyvia_data

def find_mitigation(lookupitems):
    mitigation_data = {}
    no_con_mit_count = 0
    vendors, affected_products, problem_types, targets, impacts, prevents, p_strategies, p_details = [],[],[],[],[],[],[],[]
    for cve_id in lookupitems:
        cyvia_data = find_cve_info(cve_id)
        # print(json.dumps(cyvia_data, indent=2))
        if cyvia_data[cve_id] == "No consequences or mitigation information found": 
            no_con_mit_count+=1
            continue
        
        if 'vendor' in cyvia_data[cve_id]: 
            if (cyvia_data[cve_id]['vendor'] != 'n/a'): 
                vendors.append(cyvia_data[cve_id]['vendor'])
        if 'affected_product' in cyvia_data[cve_id]: 
            if (cyvia_data[cve_id]['affected_product'] != 'n/a'): 
                affected_products.append(cyvia_data[cve_id]['affected_product'])
        if 'problem_type(s)' in cyvia_data[cve_id]: 
            if (cyvia_data[cve_id]['problem_type(s)'] != ['n/a']): 
                problem_types.extend(cyvia_data[cve_id]['problem_type(s)'])

        if 'Target(T)' in cyvia_data[cve_id]: targets.extend(cyvia_data[cve_id]['Target(T)'])
        if 'T_Impact(TI)' in cyvia_data[cve_id]: impacts.extend(cyvia_data[cve_id]['T_Impact(TI)'])
        if 'Prevent(P)' in cyvia_data[cve_id]: prevents.extend(cyvia_data[cve_id]['Prevent(P)'])
        if 'P_Strategy(PS)' in cyvia_data[cve_id]: p_strategies.extend(cyvia_data[cve_id]['P_Strategy(PS)'])
        if 'PS_Details' in cyvia_data[cve_id]: p_details.extend(cyvia_data[cve_id]['PS_Details'])
    
    print('\nVendors:', [*set(vendors)])
    print('\nAffected Products:', [*set(affected_products)])
    print('\nProblem Types:', [*set(problem_types)])
    
    print('\nTargets:', [*set(targets)])
    print('\nImpacts:', [*set(impacts)])
    print('\nPreventions(P):', [*set(prevents)])
    print('\nP_Strategies(PS):', [*set(p_strategies)])
    print('\nNo con and mit found for', no_con_mit_count, 'CVE(s). \nPreparing prevention strategies...')
    
    # summarize PS_details
    p_dets_short = []
    ps_dets = [*set(p_details)]
    for i in range(len(ps_dets)):
        x = text_summarize(ps_dets[i], 0.25) # summarize ratio 25%
        if x != '': p_dets_short.append(x) # if text is long enough and short summary is generated
        else: p_dets_short.append(ps_dets[i])    
    print('\nPS_Details:', *p_dets_short, sep = '\n')

** Server (3.1.1) up and running!



In [21]:
x = ['CVE-2005-3008', 'CVE-2008-1885', 'CVE-2001-1309', 'CVE-2006-0451', 'CVE-2022-21668', 'CVE-2008-1721', 'CVE-2015-5216', 'CVE-2004-0184', 'CVE-2017-1000433', 'CVE-2013-7061', 'CVE-2022-36099', 'CVE-2019-4000', 'CVE-2021-37678', 'CVE-2018-19646', 'CVE-2022-31571', 'CVE-2020-11057', 'CVE-2017-13106', 'CVE-2020-12608', 'CVE-2019-9740', 'CVE-2012-5578', 'CVE-2021-29539', 'CVE-2011-4104', 'CVE-2022-40426', 'CVE-2022-24065', 'CVE-2007-4593', 'CVE-2021-32618', 'CVE-2013-4346', 'CVE-2012-5499', 'CVE-2019-16935', 'CVE-2022-35873', 'CVE-2021-28861', 'CVE-2014-0105', 'CVE-2011-3378', 'CVE-2007-6015', 'CVE-2007-1359', 'CVE-2022-43304', 'CVE-2022-38882', 'CVE-2022-27177', 'CVE-2006-4980', 'CVE-2022-32151', 'CVE-2004-2629', 'CVE-2022-45061', 'CVE-2001-1311', 'CVE-2021-42771', 'CVE-2011-4355', 'CVE-2010-3081', 'CVE-2022-0391', 'CVE-2013-4409', 'CVE-2022-23594', 'CVE-2019-15795', 'CVE-2022-31518', 'CVE-2022-22817', 'CVE-2004-0218', 'CVE-2014-0472', 'CVE-2010-3494', 'CVE-2018-15747', 'CVE-2021-41848', 'CVE-2019-16056', 'CVE-2019-2435', 'CVE-2010-1449', 'CVE-2016-9259', 'CVE-2016-6580', 'CVE-2019-19274', 'CVE-2022-41381', 'CVE-2022-41380', 'CVE-2011-4211', 'CVE-2020-14343', 'CVE-2019-9636', 'CVE-2008-6547', 'CVE-2020-12003', 'CVE-2019-12408', 'CVE-2022-36027', 'CVE-2004-0220', 'CVE-2019-13597', 'CVE-2010-2089', 'CVE-2022-42039', 'CVE-2019-11236', 'CVE-2010-3493', 'CVE-2022-41383', 'CVE-2021-39182', 'CVE-2019-5629', 'CVE-2021-32633', 'CVE-2016-9910', 'CVE-2013-0294', 'CVE-2022-41931', 'CVE-2022-30298', 'CVE-2018-1000802', 'CVE-2011-1157', 'CVE-2014-7185', 'CVE-2006-4711', 'CVE-2022-42965', 'CVE-2011-4212', 'CVE-2011-4617', 'CVE-2020-25619', 'CVE-2014-3598', 'CVE-2020-12001', 'CVE-2019-11324', 'CVE-2014-1604', 'CVE-2009-3720', 'CVE-2013-4347', 'CVE-2021-41213', 'CVE-2020-5740', 'CVE-2021-33509', 'CVE-2016-10651', 'CVE-2007-5741', 'CVE-2019-7411', 'CVE-2016-0484', 'CVE-2019-11340', 'CVE-2019-9948', 'CVE-2022-44900', 'CVE-2016-4560', 'CVE-2006-0468', 'CVE-2015-1326', 'CVE-2011-1158', 'CVE-2016-7914', 'CVE-2006-1542', 'CVE-2012-3443', 'CVE-2017-12301', 'CVE-2011-1521', 'CVE-2015-1852', 'CVE-2008-6903', 'CVE-2016-1580', 'CVE-2021-36667', 'CVE-2014-1932', 'CVE-2012-5577', 'CVE-2008-5104', 'CVE-2001-1310', 'CVE-2020-10735', 'CVE-2013-1888', 'CVE-2011-4954', 'CVE-2020-5765', 'CVE-2003-1113', 'CVE-2022-21728', 'CVE-2012-5825', 'CVE-2006-2073', 'CVE-2020-26268', 'CVE-2013-5942', 'CVE-2021-28957', 'CVE-2021-32811', 'CVE-2020-27589', 'CVE-2012-1502', 'CVE-2022-29216', 'CVE-2012-0860', 'CVE-2012-2921', 'CVE-2018-1000807', 'CVE-2015-0693', 'CVE-2005-4321', 'CVE-2021-40828', 'CVE-2022-44561', 'CVE-2020-24583', 'CVE-2015-1777', 'CVE-2014-3137', 'CVE-2008-4394', 'CVE-2021-29513', 'CVE-2009-0315', 'CVE-2021-21274', 'CVE-2022-38885', 'CVE-2020-11999', 'CVE-2019-12573', 'CVE-2008-0982', 'CVE-2022-31255', 'CVE-2021-38305', 'CVE-2022-29217', 'CVE-2021-32052', 'CVE-2009-2940', 'CVE-2021-21394', 'CVE-2021-34363', 'CVE-2020-25659', 'CVE-2021-26551', 'CVE-2020-1192', 'CVE-2022-41386', 'CVE-2019-18874', 'CVE-2013-7338', 'CVE-2008-3949', 'CVE-2022-44048', 'CVE-2021-42561', 'CVE-2014-2331', 'CVE-2007-2052', 'CVE-2007-4559', 'CVE-2013-7040', 'CVE-2018-18254', 'CVE-2022-27271', 'CVE-2008-1387', 'CVE-2015-5652', 'CVE-2022-30034', 'CVE-2003-0819', 'CVE-2017-7235', 'CVE-2021-46823', 'CVE-2013-1752', 'CVE-2007-2877', 'CVE-2022-35871', 'CVE-2008-5985', 'CVE-2019-10160', 'CVE-2002-0131', 'CVE-2022-21866', 'CVE-2011-1156', 'CVE-2012-1575', 'CVE-2013-5106', 'CVE-2009-4134', 'CVE-2020-14422', 'CVE-2002-0013', 'CVE-2011-4944', 'CVE-2021-28363', 'CVE-2006-4760', 'CVE-2014-1624', 'CVE-2022-35861', 'CVE-2014-2667', 'CVE-2014-1934', 'CVE-2016-1000110', 'CVE-2019-9189', 'CVE-2010-2477', 'CVE-2018-5773', 'CVE-2022-44049', 'CVE-2019-12087', 'CVE-2019-5010', 'CVE-2003-0564', 'CVE-2020-15142', 'CVE-2008-3142', 'CVE-2019-9947', 'CVE-2006-2075', 'CVE-2002-1119', 'CVE-2002-1359', 'CVE-2015-2674', 'CVE-2007-6515', 'CVE-2020-5252', 'CVE-2017-1000158', 'CVE-2008-1886', 'CVE-2007-5137', 'CVE-2020-11888', 'CVE-2008-4108', 'CVE-2022-41382', 'CVE-2022-31575', 'CVE-2013-2167', 'CVE-2013-1753', 'CVE-2004-2680', 'CVE-2017-17522', 'CVE-2019-17514', 'CVE-2018-14649', 'CVE-2022-31177', 'CVE-2022-38886', 'CVE-2009-0316', 'CVE-2022-40809', 'CVE-2022-31117', 'CVE-2021-3572', 'CVE-2022-0718', 'CVE-2018-20061', 'CVE-2017-1002150', 'CVE-2008-1372', 'CVE-2020-10289', 'CVE-2010-0668', 'CVE-2007-1657', 'CVE-2022-40427', 'CVE-2008-5983', 'CVE-2021-29614', 'CVE-2022-30877', 'CVE-2009-0317', 'CVE-2014-1938', 'CVE-2019-9851', 'CVE-2013-4367', 'CVE-2016-5598', 'CVE-2017-2810', 'CVE-2020-13124', 'CVE-2022-38887', 'CVE-2009-0668', 'CVE-2022-22846', 'CVE-2005-3670', 'CVE-2014-1830', 'CVE-2014-0161', 'CVE-2018-16168', 'CVE-2010-4340', 'CVE-2003-1111', 'CVE-2014-7143', 'CVE-2022-42044', 'CVE-2009-3695', 'CVE-2019-19911', 'CVE-2022-21699', 'CVE-2012-4406', 'CVE-2017-6900', 'CVE-2005-2483', 'CVE-2021-21239', 'CVE-2022-23472', 'CVE-2016-7036', 'CVE-2005-3667', 'CVE-2022-30284', 'CVE-2001-1321', 'CVE-2004-0150', 'CVE-2021-36359', 'CVE-2015-1950', 'CVE-2022-31516', 'CVE-2021-29921', 'CVE-2022-29202', 'CVE-2008-2316', 'CVE-2004-0222', 'CVE-2022-41607', 'CVE-2016-1000029', 'CVE-2014-1929', 'CVE-2012-0215', 'CVE-2001-1318', 'CVE-2006-2444', 'CVE-2008-4126', 'CVE-2021-43572', 'CVE-2003-1108', 'CVE-2020-13912', 'CVE-2018-7756', 'CVE-2006-2076', 'CVE-2021-40830', 'CVE-2017-13718', 'CVE-2017-18207', 'CVE-2011-4103', 'CVE-2022-24801', 'CVE-2018-1000808', 'CVE-2001-1316', 'CVE-2021-29510', 'CVE-2022-43754', 'CVE-2003-1110', 'CVE-2021-1585', 'CVE-2013-6491', 'CVE-2017-5018', 'CVE-2022-40424', 'CVE-2022-39254', 'CVE-2014-1928', 'CVE-2018-0580', 'CVE-2013-7459', 'CVE-2016-10561', 'CVE-2001-1314', 'CVE-2001-1375', 'CVE-2017-12340', 'CVE-2022-21716', 'CVE-2005-4570', 'CVE-2019-7537', 'CVE-2022-44054', 'CVE-2014-1933', 'CVE-2014-8165', 'CVE-2021-21240', 'CVE-2021-40829', 'CVE-2013-1068', 'CVE-2015-3220', 'CVE-2019-3962', 'CVE-2022-40430', 'CVE-2012-5494', 'CVE-2006-1095', 'CVE-2002-1357', 'CVE-2011-4642', 'CVE-2014-7280', 'CVE-2022-28802', 'CVE-2018-14647', 'CVE-2016-8742', 'CVE-2009-4081', 'CVE-2022-40429', 'CVE-2006-2298', 'CVE-2009-0131', 'CVE-2005-3669', 'CVE-2020-28402', 'CVE-2017-14483', 'CVE-2006-0647', 'CVE-2020-26263', 'CVE-2019-20907', 'CVE-2021-33430', 'CVE-2005-2491', 'CVE-2020-29396', 'CVE-2014-5453', 'CVE-2003-0633', 'CVE-2022-41909', 'CVE-2010-4450', 'CVE-2015-0692', 'CVE-2021-21393', 'CVE-2022-42969', 'CVE-2020-13258', 'CVE-2022-33684', 'CVE-2005-3673', 'CVE-2012-5659', 'CVE-2008-2315', 'CVE-2022-43753', 'CVE-2013-1895', 'CVE-2020-13757', 'CVE-2003-0973', 'CVE-2022-0157', 'CVE-2005-4465', 'CVE-2009-0367', 'CVE-2013-1665', 'CVE-2022-40431', 'CVE-2004-0096', 'CVE-2016-0822', 'CVE-2022-42042', 'CVE-2020-8492', 'CVE-2020-24584', 'CVE-2010-0211', 'CVE-2008-6536', 'CVE-2020-27619', 'CVE-2021-32674', 'CVE-2016-6716', 'CVE-2016-9260', 'CVE-2021-4189', 'CVE-2020-5215', 'CVE-2012-4520', 'CVE-2022-36070', 'CVE-2022-40810', 'CVE-2019-11507', 'CVE-2010-1666', 'CVE-2008-0553', 'CVE-2019-12805', 'CVE-2017-16763', 'CVE-2017-3311', 'CVE-2006-0453', 'CVE-2013-7323', 'CVE-2022-22984', 'CVE-2017-5524', 'CVE-2019-10633', 'CVE-2005-0852', 'CVE-2012-5487', 'CVE-2004-1491', 'CVE-2008-5103', 'CVE-2022-29205', 'CVE-2021-44255', 'CVE-2020-25596', 'CVE-2013-5654', 'CVE-2018-0015', 'CVE-2014-6448', 'CVE-2019-3923', 'CVE-2019-9855', 'CVE-2021-33026', 'CVE-2012-2374', 'CVE-2015-3206', 'CVE-2013-1445', 'CVE-2022-40425', 'CVE-2015-2316', 'CVE-2021-32807', 'CVE-2017-9807', 'CVE-2005-3672', 'CVE-2017-3590', 'CVE-2021-32831', 'CVE-2022-42041', 'CVE-2005-0088', 'CVE-2022-40811', 'CVE-2018-16858', 'CVE-2013-5123', 'CVE-2022-0196', 'CVE-2018-12175', 'CVE-2020-15271', 'CVE-2022-41387', 'CVE-2022-40428', 'CVE-2022-31116', 'CVE-2022-36024', 'CVE-2014-1838', 'CVE-2019-17626', 'CVE-2014-0485', 'CVE-2017-10803', 'CVE-2021-41131', 'CVE-2010-1450', 'CVE-2005-3733', 'CVE-2008-6954', 'CVE-2020-11073', 'CVE-2003-1114', 'CVE-2010-1634', 'CVE-2012-0878', 'CVE-2016-3180', 'CVE-2001-1317', 'CVE-2022-31124', 'CVE-2021-21238', 'CVE-2006-2078', 'CVE-2022-40806', 'CVE-2022-24859', 'CVE-2005-3915', 'CVE-2013-6418', 'CVE-2009-3578', 'CVE-2020-5741', 'CVE-2004-0183', 'CVE-2008-1837', 'CVE-2018-14572', 'CVE-2012-2135', 'CVE-2008-7144', 'CVE-2001-0975', 'CVE-2021-29239', 'CVE-2022-28696', 'CVE-2021-41281', 'CVE-2005-3668', 'CVE-2018-20325', 'CVE-2017-16618', 'CVE-2014-3593', 'CVE-2013-7489', 'CVE-2022-41887', 'CVE-2021-21273', 'CVE-2021-40831', 'CVE-2020-26116', 'CVE-2018-6012', 'CVE-2004-1050', 'CVE-2020-15101', 'CVE-2014-7144', 'CVE-2020-15720', 'CVE-2018-10903', 'CVE-2022-31471', 'CVE-2021-31997', 'CVE-2011-4357', 'CVE-2021-21317', 'CVE-2006-0645', 'CVE-2006-0452', 'CVE-2011-1015', 'CVE-2019-13611', 'CVE-2012-2146', 'CVE-2005-3666', 'CVE-2020-7212', 'CVE-2022-1941', 'CVE-2012-0845', 'CVE-2001-1313', 'CVE-2022-33977', 'CVE-2016-5636', 'CVE-2008-2020', 'CVE-2008-0981', 'CVE-2020-35766', 'CVE-2022-41385', 'CVE-2016-1853', 'CVE-2022-0571', 'CVE-2022-42036', 'CVE-2019-17526', 'CVE-2016-6581', 'CVE-2005-2875', 'CVE-2014-4650', 'CVE-2020-26244', 'CVE-2021-41168', 'CVE-2022-23586', 'CVE-2022-38884', 'CVE-2005-3768', 'CVE-2022-24761', 'CVE-2019-3575', 'CVE-2019-15796', 'CVE-2020-15348', 'CVE-2010-3964', 'CVE-2013-4238', 'CVE-2019-16729', 'CVE-2016-1000028', 'CVE-2006-0900', 'CVE-2019-17019', 'CVE-2008-5984', 'CVE-2010-3858', 'CVE-2014-3539', 'CVE-2008-4099', 'CVE-2002-1360', 'CVE-2015-1341', 'CVE-2012-6081', 'CVE-2018-0023', 'CVE-2013-1055', 'CVE-2006-2074', 'CVE-2020-12005', 'CVE-2014-6262', 'CVE-2015-5242', 'CVE-2007-6531', 'CVE-2016-0489', 'CVE-2019-12410', 'CVE-2001-1320', 'CVE-2020-25658', 'CVE-2022-44053', 'CVE-2012-0861', 'CVE-2021-39317', 'CVE-2021-3177', 'CVE-2004-0097', 'CVE-2006-2446', 'CVE-2020-10799', 'CVE-2015-3446', 'CVE-2014-4861', 'CVE-2021-42343', 'CVE-2011-0725', 'CVE-2017-5179', 'CVE-2020-24715', 'CVE-2022-40807', 'CVE-2018-7889', 'CVE-2019-7617', 'CVE-2013-2072', 'CVE-2012-4245', 'CVE-2021-3426', 'CVE-2008-1412', 'CVE-2022-2996', 'CVE-2014-3007', 'CVE-2005-3732', 'CVE-2022-44050', 'CVE-2022-36087', 'CVE-2010-3492', 'CVE-2022-26488', 'CVE-2022-0238', 'CVE-2019-19588', 'CVE-2005-4564', 'CVE-2010-4157', 'CVE-2020-29651', 'CVE-2018-2753', 'CVE-2006-0717', 'CVE-2014-1912', 'CVE-2020-27783', 'CVE-2013-7440', 'CVE-2008-6539', 'CVE-2004-0056', 'CVE-2021-25216', 'CVE-2018-1060', 'CVE-2019-14853', 'CVE-2020-35678', 'CVE-2021-34552', 'CVE-2021-25322', 'CVE-2017-12612', 'CVE-2018-1061', 'CVE-2021-0979', 'CVE-2021-28359', 'CVE-2020-26943', 'CVE-2020-1747', 'CVE-2012-3458', 'CVE-2021-21236', 'CVE-2019-14347', 'CVE-2016-4043', 'CVE-2016-5699', 'CVE-2003-0565', 'CVE-2020-27384', 'CVE-2017-6543', 'CVE-2011-4783', 'CVE-2011-3587', 'CVE-2013-1229', 'CVE-2013-4867', 'CVE-2022-36834', 'CVE-2002-1358', 'CVE-2010-2235', 'CVE-2009-5010', 'CVE-2015-5306', 'CVE-2022-36069', 'CVE-2019-3558', 'CVE-2015-6531', 'CVE-2012-5506', 'CVE-2006-0151', 'CVE-2020-26222', 'CVE-2022-39227', 'CVE-2022-42038', 'CVE-2016-0482', 'CVE-2019-13404', 'CVE-2005-4566', 'CVE-2012-1176', 'CVE-2021-42576', 'CVE-2014-3589', 'CVE-2020-8950', 'CVE-2022-36065', 'CVE-2020-16977', 'CVE-2013-2132', 'CVE-2017-16616', 'CVE-2001-0974', 'CVE-2002-0012', 'CVE-2016-2542', 'CVE-2016-1494', 'CVE-2016-2533', 'CVE-2006-2077', 'CVE-2009-3729', 'CVE-2006-0718', 'CVE-2012-3444', 'CVE-2006-2240', 'CVE-2013-2013', 'CVE-2013-2191', 'CVE-2016-0487', 'CVE-2014-3995', 'CVE-2020-15801', 'CVE-2015-5164', 'CVE-2017-16764', 'CVE-2014-5340', 'CVE-2017-16615', 'CVE-2004-0219', 'CVE-2015-7546', 'CVE-2021-29567', 'CVE-2013-1629', 'CVE-2016-0486', 'CVE-2021-21419', 'CVE-2021-32677', 'CVE-2013-6444', 'CVE-2013-4482', 'CVE-2009-3894', 'CVE-2012-5485', 'CVE-2018-6461', 'CVE-2012-5650', 'CVE-2022-42043', 'CVE-2022-40805', 'CVE-2019-19275', 'CVE-2020-8315', 'CVE-2021-43818', 'CVE-2001-1308', 'CVE-2001-1319', 'CVE-2020-1171', 'CVE-2008-5986', 'CVE-2022-21817', 'CVE-2012-6080', 'CVE-2009-0218', 'CVE-2021-21241', 'CVE-2016-0481', 'CVE-2006-0052', 'CVE-2017-2809', 'CVE-2016-9950', 'CVE-2022-20419', 'CVE-2021-29514', 'CVE-2021-33880', 'CVE-2009-3850', 'CVE-2022-40432', 'CVE-2006-7228', 'CVE-2021-21332', 'CVE-2019-14859', 'CVE-2009-5065', 'CVE-2012-2417', 'CVE-2013-5093', 'CVE-2003-1112', 'CVE-2021-21330', 'CVE-2020-7984', 'CVE-2011-1444', 'CVE-2021-39158', 'CVE-2016-0480', 'CVE-2012-5474', 'CVE-2013-2166', 'CVE-2022-43303', 'CVE-2003-1109', 'CVE-2020-25489', 'CVE-2021-29572', 'CVE-2022-0197', 'CVE-2019-12761', 'CVE-2008-5987', 'CVE-2021-3737', 'CVE-2011-1364', 'CVE-2020-8897', 'CVE-2006-2072', 'CVE-2009-3724', 'CVE-2021-29571', 'CVE-2022-36040', 'CVE-2022-44560', 'CVE-2021-43837', 'CVE-2008-3294', 'CVE-2022-40808', 'CVE-2021-21392', 'CVE-2017-2592', 'CVE-2001-1312', 'CVE-2021-38112', 'CVE-2013-2099', 'CVE-2012-6495', 'CVE-2008-6549', 'CVE-2008-1887', 'CVE-2022-35920', 'CVE-2018-6353', 'CVE-2018-20534', 'CVE-2008-3143', 'CVE-2022-41945', 'CVE-2020-15141', 'CVE-2022-32552', 'CVE-2019-7653', 'CVE-2018-20406', 'CVE-2014-8650', 'CVE-2012-5493', 'CVE-2022-30885', 'CVE-2010-0212', 'CVE-2019-20916', 'CVE-2010-3495', 'CVE-2012-5379', 'CVE-2016-0490', 'CVE-2022-38881', 'CVE-2008-4864', 'CVE-2020-13948', 'CVE-2006-4710', 'CVE-2012-1150', 'CVE-2010-4255', 'CVE-2022-35918', 'CVE-2019-17664', 'CVE-2022-44051', 'CVE-2022-23651', 'CVE-2019-3980', 'CVE-2021-45082', 'CVE-2021-28667', 'CVE-2019-6690', 'CVE-2021-4007', 'CVE-2017-1000246', 'CVE-2004-0054', 'CVE-2015-4234', 'CVE-2011-2520', 'CVE-2021-21333', 'CVE-2022-36126', 'CVE-2018-3650', 'CVE-2015-5159', 'CVE-2022-42037', 'CVE-2016-0771', 'CVE-2012-3533', 'CVE-2007-4308', 'CVE-2020-15163', 'CVE-2018-1000117', 'CVE-2022-23599', 'CVE-2019-9674', 'CVE-2019-12576', 'CVE-2021-20135', 'CVE-2022-26236', 'CVE-2020-36242', 'CVE-2012-5822', 'CVE-2005-3291', 'CVE-2020-15207', 'CVE-2008-4863', 'CVE-2007-1253', 'CVE-2009-5042', 'CVE-2005-0089', 'CVE-2021-3733', 'CVE-2008-1679', 'CVE-2018-1000030', 'CVE-2022-41384', 'CVE-2019-5729', 'CVE-2018-18256', 'CVE-2018-17175', 'CVE-2022-42919', 'CVE-2008-1965', 'CVE-2013-1664', 'CVE-2001-1307', 'CVE-2011-4213', 'CVE-2020-5227', 'CVE-2012-5488', 'CVE-2021-32839', 'CVE-2021-43854', 'CVE-2022-2634', 'CVE-2021-40839', 'CVE-2019-20807', 'CVE-2017-0906', 'CVE-2016-9909', 'CVE-2016-4972', 'CVE-2001-0701', 'CVE-2020-13144', 'CVE-2012-5563', 'CVE-2021-29471', 'CVE-2005-3302', 'CVE-2016-9949', 'CVE-2018-17707', 'CVE-2014-1927', 'CVE-2013-5655', 'CVE-2020-12878', 'CVE-2021-29548', 'CVE-2019-6446', 'CVE-2005-3985', 'CVE-2020-9306', 'CVE-2013-2104', 'CVE-2005-3674', 'CVE-2014-1829', 'CVE-2004-2758', 'CVE-2020-27351', 'CVE-2022-43306', 'CVE-2019-9854', 'CVE-2015-7489', 'CVE-2022-39280', 'CVE-2014-9365', 'CVE-2017-20052', 'CVE-2014-4980', 'CVE-2022-41908', 'CVE-2021-23336', 'CVE-2017-9462', 'CVE-2014-9090', 'CVE-2015-0290', 'CVE-2013-2962', 'CVE-2019-15903', 'CVE-2017-7262', 'CVE-2019-1727', 'CVE-2021-41125', 'CVE-2005-2966', 'CVE-2011-4407', 'CVE-2001-1306', 'CVE-2014-2967', 'CVE-2021-29421', 'CVE-2015-4604', 'CVE-2022-27237', 'CVE-2009-0314', 'CVE-2016-9015', 'CVE-2019-9850', 'CVE-2008-0299', 'CVE-2009-0318', 'CVE-2022-43305', 'CVE-2019-18355', 'CVE-2020-13388', 'CVE-2016-0772', 'CVE-2004-1457', 'CVE-2021-36829', 'CVE-2021-41250', 'CVE-2019-10138', 'CVE-2014-4616', 'CVE-2013-1909', 'CVE-2020-15251', 'CVE-2022-38880', 'CVE-2022-41934', 'CVE-2010-0395', 'CVE-2020-7645', 'CVE-2022-42040', 'CVE-2018-20852', 'CVE-2016-0485', 'CVE-2022-38883', 'CVE-2012-5495', 'CVE-2019-18348', 'CVE-2019-10099', 'CVE-2010-2480', 'CVE-2018-6393', 'CVE-2019-15790', 'CVE-2009-4924', 'CVE-2022-44052', 'CVE-2006-1646', 'CVE-2014-3146', 'CVE-2014-3994', 'CVE-2022-30688', 'CVE-2006-4762', 'CVE-2006-4761', 'CVE-2021-39271', 'CVE-2009-2205', 'CVE-2022-21712', 'CVE-2011-4137', 'CVE-2013-2131', 'CVE-2019-19875', 'CVE-2015-4605', 'CVE-2012-4571', 'CVE-2015-20107', 'CVE-2015-4231', 'CVE-2006-1268', 'CVE-2013-4111', 'CVE-2018-18074', 'CVE-2019-9852', 'CVE-2022-27896', 'CVE-2014-1839', 'CVE-2013-4245', 'CVE-2011-4940', 'CVE-2019-9848', 'CVE-2008-5031', 'CVE-2022-30286', 'CVE-2005-3671', 'CVE-2001-1315', 'CVE-2020-15523', 'CVE-2020-15415', 'CVE-2013-6396', 'CVE-2008-5102', 'CVE-2021-26605', 'CVE-2022-36100', 'CVE-2005-4151', 'CVE-2003-1115', 'CVE-2008-3144', 'CVE-2016-2166', 'CVE-2008-0980', 'CVE-2005-4565', 'CVE-2017-4932', 'CVE-2022-31015', 'CVE-2017-1000483', 'CVE-2004-0221', 'CVE-2022-40812', 'CVE-2020-15193', 'CVE-2016-5851', 'CVE-2002-0185', 'CVE-2021-29512', 'CVE-2021-33571', 'CVE-2007-4965']    
# find_mitigation(x[185:190])
# find_mitigation(x[0:200])
# find_mitigation(x[0:190])
find_mitigation(x[4:10])
# find_mitigation(['CVE-2021-46823'])  


Vendors: ['pypa']

Affected Products: ['pipenv', 'Ipsilon']

Problem Types: ['CWE-791: Incomplete Filtering of Special Elements', 'Cross-Site Scripting', "CWE-77: Improper Neutralization of Special Elements used in a Command ('Command Injection')", 'CWE-20: Improper Input Validation', "CWE-78: Improper Neutralization of Special Elements used in an OS Command ('OS Command Injection')", 'CWE-427: Uncontrolled Search Path Element']

Targets: ['Integrity', 'Other', 'Availability', 'Access Control', 'Confidentiality']

Impacts: ['DoS: Resource Consumption (Memory)', 'Gain Privileges or Assume Identity', 'Read Files or Directories', 'Read Memory', 'Read Application Data', 'Unexpected State', 'DoS: Resource Consumption (CPU)', 'DoS: Crash, Exit, or Restart', 'Modify Memory', 'Execute Unauthorized Code or Commands', 'Bypass Protection Mechanism', 'Quality Degradation']

Preventions(P): ['Implementation Architecture and Design', 'Architecture and Design Implementation', 'Operation', 'Architect

In [22]:
x = {"containers":{"cna":{"affected":[{"product":"pipenv","vendor":"pypa","versions":[{"status":"affected","version":">= 2018.10.9, < 2022.1.8"}]}],"descriptions":[{"lang":"en","value":"pipenv is a Python development workflow tool. Starting with version 2018.10.9 and prior to version 2022.1.8, a flaw in pipenv's parsing of requirements files allows an attacker to insert a specially crafted string inside a comment anywhere within a requirements.txt file, which will cause victims who use pipenv to install the requirements file to download dependencies from a package index server controlled by the attacker. By embedding malicious code in packages served from their malicious index server, the attacker can trigger arbitrary remote code execution (RCE) on the victims' systems. If an attacker is able to hide a malicious `--index-url` option in a requirements file that a victim installs with pipenv, the attacker can embed arbitrary malicious code in packages served from their malicious index server that will be executed on the victim's host during installation (remote code execution/RCE). When pip installs from a source distribution, any code in the setup.py is executed by the install process. This issue is patched in version 2022.1.8. The GitHub Security Advisory contains more information about this vulnerability."}],"metrics":[{"cvssV3_1":{"attackComplexity":"HIGH","attackVector":"NETWORK","availabilityImpact":"HIGH","baseScore":8,"baseSeverity":"HIGH","confidentialityImpact":"HIGH","integrityImpact":"HIGH","privilegesRequired":"LOW","scope":"CHANGED","userInteraction":"REQUIRED","vectorString":"CVSS:3.1/AV:N/AC:H/PR:L/UI:R/S:C/C:H/I:H/A:H","version":"3.1"}}],"problemTypes":[{"descriptions":[{"cweId":"CWE-20","description":"CWE-20: Improper Input Validation","lang":"en","type":"CWE"}]},{"descriptions":[{"cweId":"CWE-77","description":"CWE-77: Improper Neutralization of Special Elements used in a Command ('Command Injection')","lang":"en","type":"CWE"}]},{"descriptions":[{"cweId":"CWE-78","description":"CWE-78: Improper Neutralization of Special Elements used in an OS Command ('OS Command Injection')","lang":"en","type":"CWE"}]},{"descriptions":[{"cweId":"CWE-427","description":"CWE-427: Uncontrolled Search Path Element","lang":"en","type":"CWE"}]},{"descriptions":[{"cweId":"CWE-791","description":"CWE-791: Incomplete Filtering of Special Elements","lang":"en","type":"CWE"}]}],"providerMetadata":{"dateUpdated":"2022-03-26T17:06:41","orgId":"a0819718-46f1-4df5-94e2-005712e83aaa","shortName":"GitHub_M"},"references":[{"tags":["x_refsource_CONFIRM"],"url":"https://github.com/pypa/pipenv/security/advisories/GHSA-qc9x-gjcv-465w"},{"tags":["x_refsource_MISC"],"url":"https://github.com/pypa/pipenv/commit/439782a8ae36c4762c88e43d5f0d8e563371b46f"},{"tags":["x_refsource_MISC"],"url":"https://github.com/pypa/pipenv/releases/tag/v2022.1.8"},{"name":"FEDORA-2022-77ce20f03a","tags":["vendor-advisory","x_refsource_FEDORA"],"url":"https://lists.fedoraproject.org/archives/list/package-announce%40lists.fedoraproject.org/message/QHQRIWKDP3SVJABAPEXBIQPKDI6UP7G4/"},{"name":"FEDORA-2022-508e460384","tags":["vendor-advisory","x_refsource_FEDORA"],"url":"https://lists.fedoraproject.org/archives/list/package-announce%40lists.fedoraproject.org/message/KCROBYHUS6DKQPCXBRPCZ5CDBNQTYAWT/"},{"name":"FEDORA-2022-0d007466b3","tags":["vendor-advisory","x_refsource_FEDORA"],"url":"https://lists.fedoraproject.org/archives/list/package-announce%40lists.fedoraproject.org/message/56HBA3EOSLEDNCCBJVHE6DO34P56EOUM/"}],"source":{"advisory":"GHSA-qc9x-gjcv-465w","discovery":"UNKNOWN"},"title":"Pipenv's requirements.txt parsing allows malicious index url in comments","x_legacyV4Record":{"CVE_data_meta":{"ASSIGNER":"security-advisories@github.com","ID":"CVE-2022-21668","STATE":"PUBLIC","TITLE":"Pipenv's requirements.txt parsing allows malicious index url in comments"},"affects":{"vendor":{"vendor_data":[{"product":{"product_data":[{"product_name":"pipenv","version":{"version_data":[{"version_value":">= 2018.10.9, < 2022.1.8"}]}}]},"vendor_name":"pypa"}]}},"data_format":"MITRE","data_type":"CVE","data_version":"4.0","description":{"description_data":[{"lang":"eng","value":"pipenv is a Python development workflow tool. Starting with version 2018.10.9 and prior to version 2022.1.8, a flaw in pipenv's parsing of requirements files allows an attacker to insert a specially crafted string inside a comment anywhere within a requirements.txt file, which will cause victims who use pipenv to install the requirements file to download dependencies from a package index server controlled by the attacker. By embedding malicious code in packages served from their malicious index server, the attacker can trigger arbitrary remote code execution (RCE) on the victims' systems. If an attacker is able to hide a malicious `--index-url` option in a requirements file that a victim installs with pipenv, the attacker can embed arbitrary malicious code in packages served from their malicious index server that will be executed on the victim's host during installation (remote code execution/RCE). When pip installs from a source distribution, any code in the setup.py is executed by the install process. This issue is patched in version 2022.1.8. The GitHub Security Advisory contains more information about this vulnerability."}]},"impact":{"cvss":{"attackComplexity":"HIGH","attackVector":"NETWORK","availabilityImpact":"HIGH","baseScore":8,"baseSeverity":"HIGH","confidentialityImpact":"HIGH","integrityImpact":"HIGH","privilegesRequired":"LOW","scope":"CHANGED","userInteraction":"REQUIRED","vectorString":"CVSS:3.1/AV:N/AC:H/PR:L/UI:R/S:C/C:H/I:H/A:H","version":"3.1"}},"problemtype":{"problemtype_data":[{"description":[{"lang":"eng","value":"CWE-20: Improper Input Validation"}]},{"description":[{"lang":"eng","value":"CWE-77: Improper Neutralization of Special Elements used in a Command ('Command Injection')"}]},{"description":[{"lang":"eng","value":"CWE-78: Improper Neutralization of Special Elements used in an OS Command ('OS Command Injection')"}]},{"description":[{"lang":"eng","value":"CWE-427: Uncontrolled Search Path Element"}]},{"description":[{"lang":"eng","value":"CWE-791: Incomplete Filtering of Special Elements"}]}]},"references":{"reference_data":[{"name":"https://github.com/pypa/pipenv/security/advisories/GHSA-qc9x-gjcv-465w","refsource":"CONFIRM","url":"https://github.com/pypa/pipenv/security/advisories/GHSA-qc9x-gjcv-465w"},{"name":"https://github.com/pypa/pipenv/commit/439782a8ae36c4762c88e43d5f0d8e563371b46f","refsource":"MISC","url":"https://github.com/pypa/pipenv/commit/439782a8ae36c4762c88e43d5f0d8e563371b46f"},{"name":"https://github.com/pypa/pipenv/releases/tag/v2022.1.8","refsource":"MISC","url":"https://github.com/pypa/pipenv/releases/tag/v2022.1.8"},{"name":"FEDORA-2022-77ce20f03a","refsource":"FEDORA","url":"https://lists.fedoraproject.org/archives/list/package-announce@lists.fedoraproject.org/message/QHQRIWKDP3SVJABAPEXBIQPKDI6UP7G4/"},{"name":"FEDORA-2022-508e460384","refsource":"FEDORA","url":"https://lists.fedoraproject.org/archives/list/package-announce@lists.fedoraproject.org/message/KCROBYHUS6DKQPCXBRPCZ5CDBNQTYAWT/"},{"name":"FEDORA-2022-0d007466b3","refsource":"FEDORA","url":"https://lists.fedoraproject.org/archives/list/package-announce@lists.fedoraproject.org/message/56HBA3EOSLEDNCCBJVHE6DO34P56EOUM/"}]},"source":{"advisory":"GHSA-qc9x-gjcv-465w","discovery":"UNKNOWN"}}}},"cveMetadata":{"assignerOrgId":"a0819718-46f1-4df5-94e2-005712e83aaa","assignerShortName":"GitHub_M","cveId":"CVE-2022-21668","datePublished":"2022-01-10T20:20:16","dateReserved":"2021-11-16T00:00:00","dateUpdated":"2022-03-26T17:06:41","state":"PUBLISHED"},"dataType":"CVE_RECORD","dataVersion":"5.0"}
print(json.dumps(x, indent=2))

{
  "containers": {
    "cna": {
      "affected": [
        {
          "product": "pipenv",
          "vendor": "pypa",
          "versions": [
            {
              "status": "affected",
              "version": ">= 2018.10.9, < 2022.1.8"
            }
          ]
        }
      ],
      "descriptions": [
        {
          "lang": "en",
          "value": "pipenv is a Python development workflow tool. Starting with version 2018.10.9 and prior to version 2022.1.8, a flaw in pipenv's parsing of requirements files allows an attacker to insert a specially crafted string inside a comment anywhere within a requirements.txt file, which will cause victims who use pipenv to install the requirements file to download dependencies from a package index server controlled by the attacker. By embedding malicious code in packages served from their malicious index server, the attacker can trigger arbitrary remote code execution (RCE) on the victims' systems. If an attacker is able to hide a 

In [23]:
a = [
"Use a language that does not allow this weakness to occur or provides constructs that make this weakness easier to avoid. For example, many languages that perform their own memory management, such as Java and Perl, are not subject to buffer overflows. Other languages, such as Ada and C#, typically provide overflow protection, but the protection can be disabled by the programmer. Be wary that a language's interface to native code may still be subject to overflows, even if the language itself is theoretically safe.",
            
"Use a vetted library or framework that does not allow this weakness to occur or provides constructs that make this weakness easier to avoid. Examples include the Safe C String Library (SafeStr) by Messier and Viega [REF-57], and the Strsafe.h library from Microsoft [REF-56]. These libraries provide safer versions of overflow-prone string-handling functions.",

"Run or compile the software using features or extensions that automatically provide a protection mechanism that mitigates or eliminates buffer overflows. For example, certain compilers and extensions provide automatic buffer overflow detection mechanisms that are built into the compiled code. Examples include the Microsoft Visual Studio /GS flag, Fedora/Red Hat FORTIFY_SOURCE GCC flag, StackGuard, and ProPolice.",

"Consider adhering to the following rules when allocating and managing an application's memory: Double check that the buffer is as large as specified. When using functions that accept a number of bytes to copy, such as strncpy(), be aware that if the destination buffer size is equal to the source buffer size, it may not NULL-terminate the string. Check buffer boundaries if accessing the buffer in a loop and make sure there is no danger of writing past the allocated space. If necessary, truncate all input strings to a reasonable length before passing them to the copy and concatenation functions.",

"Replace unbounded copy functions with analogous functions that support length arguments, such as strcpy with strncpy. Create these if they are not available.",

"Run or compile the software using features or extensions that randomly arrange the positions of a program's executable and libraries in memory. Because this makes the addresses unpredictable, it can prevent an attacker from reliably jumping to exploitable code. Examples include Address Space Layout Randomization (ASLR) [REF-58] [REF-60] and Position-Independent Executables (PIE) [REF-64].",

"Use a CPU and operating system that offers Data Execution Protection (NX) or its equivalent [REF-60] [REF-61]."

"Avoid making conversion between numeric types. Always check for the allowed ranges.",

"Use a vetted library or framework that does not allow this weakness to occur or provides constructs that make this weakness easier to avoid. Examples of libraries and frameworks that make it easier to generate properly encoded output include Microsoft's Anti-XSS library, the OWASP ESAPI Encoding module, and Apache Wicket.",

"For any security checks that are performed on the client side, ensure that these checks are duplicated on the server side, in order to avoid CWE-602. Attackers can bypass the client-side checks by modifying values after the checks have been performed, or by changing the client to remove the client-side checks entirely. Then, these modified values would be submitted to the server.",

"If available, use structured mechanisms that automatically enforce the separation between data and code. These mechanisms may be able to provide the relevant quoting, encoding, and validation automatically, instead of relying on the developer to provide this capability at every point where output is generated.",

"When the set of acceptable objects, such as filenames or URLs, is limited or known, create a mapping from a set of fixed input values (such as numeric IDs) to the actual filenames or URLs, and reject all other inputs.",

"Understand the context in which your data will be used and the encoding that will be expected. This is especially important when transmitting data between different components, or when generating outputs that can contain multiple encodings at the same time, such as web pages or multi-part mail messages. Study all expected communication protocols and data representations to determine the required encoding strategies. For any data that will be output to another web page, especially any data that was received from external inputs, use the appropriate encoding on all non-alphanumeric characters. Parts of the same output document may require different encodings, which will vary depending on whether the output is in the: HTML body Element attributes (such as src=XYZ) URIs JavaScript sections Cascading Style Sheets and style property etc. Note that HTML Entity Encoding is only appropriate for the HTML body. Consult the XSS Prevention Cheat Sheet [REF-724] for more details on the types of encoding and escaping that are needed.",

"Understand all the potential areas where untrusted inputs can enter your software: parameters or arguments, cookies, anything read from the network, environment variables, reverse DNS lookups, query results, request headers, URL components, e-mail, files, filenames, databases, and any external systems that provide data to the application. Remember that such inputs may be obtained indirectly through API calls.",

"Use and specify an output encoding that can be handled by the downstream component that is reading the output. Common encodings include ISO-8859-1, UTF-7, and UTF-8. When an encoding is not specified, a downstream component may choose a different encoding, either by assuming a default encoding or automatically inferring which encoding is being used, which can be erroneous. When the encodings are inconsistent, the downstream component might treat some character or byte sequences as special, even if they are not special in the original encoding. Attackers might then be able to exploit this discrepancy and conduct injection attacks; they even might be able to bypass protection mechanisms that assume the original encoding is also being used by the downstream component. The problem of inconsistent output encodings often arises in web pages. If an encoding is not specified in an HTTP header, web browsers often guess about which encoding is being used. This can open up the browser to subtle XSS attacks.",

"With Struts, write all data from form beans with the bean's filter attribute set to true.",

"To help mitigate XSS attacks against the user's session cookie, set the session cookie to be HttpOnly. In browsers that support the HttpOnly feature (such as more recent versions of Internet Explorer and Firefox), this attribute can prevent the user's session cookie from being accessible to malicious client-side scripts that use document.cookie. This is not a complete solution, since HttpOnly is not supported by all browsers. More importantly, XMLHTTPRequest and other powerful browser technologies provide read access to HTTP headers, including the Set-Cookie header in which the HttpOnly flag is set.",

"Assume all input is malicious. Use an accept known good input validation strategy, i.e., use a list of acceptable inputs that strictly conform to specifications. Reject any input that does not strictly conform to specifications, or transform it into something that does. When performing input validation, consider all potentially relevant properties, including length, type of input, the full range of acceptable values, missing or extra inputs, syntax, consistency across related fields, and conformance to business rules. As an example of business rule logic, boat may be syntactically valid because it only contains alphanumeric characters, but it is not valid if the input is only expected to contain colors such as red or blue. Do not rely exclusively on looking for malicious or malformed inputs. This is likely to miss at least one undesirable input, especially if the code's environment changes. This can give attackers enough room to bypass the intended validation. However, denylists can be useful for detecting potential attacks or determining which inputs are so malformed that they should be rejected outright. When dynamically constructing web pages, use stringent allowlists that limit the character set based on the expected value of the parameter in the request. All input should be validated and cleansed, not just parameters that the user is supposed to specify, but all data in the request, including hidden fields, cookies, headers, the URL itself, and so forth. A common mistake that leads to continuing XSS vulnerabilities is to validate only fields that are expected to be redisplayed by the site. It is common to see data from the request that is reflected by the application server or the application that the development team did not anticipate. Also, a field that is not currently reflected may be used by a future developer. Therefore, validating ALL parts of the HTTP request is recommended. Note that proper output encoding, escaping, and quoting is the most effective solution for preventing XSS, although input validation may provide some defense-in-depth. This is because it effectively limits what will appear in output. Input validation will not always prevent XSS, especially if you are required to support free-form text fields that could contain arbitrary characters. For example, in a chat application, the heart emoticon (<3) would likely pass the validation step, since it is commonly used. However, it cannot be directly inserted into the web page because it contains the < character, which would need to be escaped or otherwise handled. In this case, stripping the < might reduce the risk of XSS, but it would produce incorrect behavior because the emoticon would not be recorded. This might seem to be a minor inconvenience, but it would be more important in a mathematical forum that wants to represent inequalities. Even if you make a mistake in your validation (such as forgetting one out of 100 input fields), appropriate encoding is still likely to protect you from injection-based attacks. As long as it is not done in isolation, input validation is still a useful technique, since it may significantly reduce your attack surface, allow you to detect some attacks, and provide other security benefits that proper encoding does not address. Ensure that you perform input validation at well-defined interfaces within the application. This will help protect the application even if a component is reused or moved elsewhere.",

"Use an application firewall that can detect attacks against this weakness. It can be beneficial in cases in which the code cannot be fixed (because it is controlled by a third party), as an emergency prevention measure while more comprehensive software assurance measures are applied, or to provide defense in depth.",

"When using PHP, configure the application so that it does not use register_globals. During implementation, develop the application so that it does not rely on this feature, but be wary of implementing a register_globals emulation that is subject to weaknesses such as CWE-95, CWE-621, and similar issues.",

"Consider using language-theoretic security (LangSec) techniques that characterize inputs using a formal language and build recognizers for that language. This effectively requires parsing to be a distinct layer that effectively enforces a boundary between raw input and internal data representations, instead of allowing parser code to be scattered throughout the program, where it could be subject to errors or inconsistencies that create weaknesses. [REF-1109] [REF-1110] [REF-1111]",

"Use an input validation framework such as Struts or the OWASP ESAPI Validation API. Note that using a framework does not automatically address all input validation problems; be mindful of weaknesses that could arise from misusing the framework itself (CWE-1173).",

"For any security checks that are performed on the client side, ensure that these checks are duplicated on the server side, in order to avoid CWE-602. Attackers can bypass the client-side checks by modifying values after the checks have been performed, or by changing the client to remove the client-side checks entirely. Then, these modified values would be submitted to the server. Even though client-side checks provide minimal benefits with respect to server-side security, they are still useful. First, they can support intrusion detection. If the server receives input that should have been rejected by the client, then it may be an indication of an attack. Second, client-side error-checking can provide helpful feedback to the user about the expectations for valid input. Third, there may be a reduction in server-side processing time for accidental input errors, although this is typically a small savings.",

"Understand all the potential areas where untrusted inputs can enter your software: parameters or arguments, cookies, anything read from the network, environment variables, reverse DNS lookups, query results, request headers, URL components, e-mail, files, filenames, databases, and any external systems that provide data to the application. Remember that such inputs may be obtained indirectly through API calls.",

"Assume all input is malicious. Use an accept known good input validation strategy, i.e., use a list of acceptable inputs that strictly conform to specifications. Reject any input that does not strictly conform to specifications, or transform it into something that does. When performing input validation, consider all potentially relevant properties, including length, type of input, the full range of acceptable values, missing or extra inputs, syntax, consistency across related fields, and conformance to business rules. As an example of business rule logic, boat may be syntactically valid because it only contains alphanumeric characters, but it is not valid if the input is only expected to contain colors such as red or blue. Do not rely exclusively on looking for malicious or malformed inputs. This is likely to miss at least one undesirable input, especially if the code's environment changes. This can give attackers enough room to bypass the intended validation. However, denylists can be useful for detecting potential attacks or determining which inputs are so malformed that they should be rejected outright.",

"When your application combines data from multiple sources, perform the validation after the sources have been combined. The individual data elements may pass the validation step but violate the intended restrictions after they have been combined.",

"Be especially careful to validate all input when invoking code that crosses language boundaries, such as from an interpreted language to native code. This could create an unexpected interaction between the language boundaries. Ensure that you are not violating any of the expectations of the language with which you are interfacing. For example, even though Java may not be susceptible to buffer overflows, providing a large argument in a call to native code might trigger an overflow.",

"Directly convert your input type into the expected data type, such as using a conversion function that translates a string into a number. After converting to the expected data type, ensure that the input's values fall within the expected range of allowable values and that multi-field consistencies are maintained.",

"Inputs should be decoded and canonicalized to the application's current internal representation before being validated (CWE-180, CWE-181). Make sure that your application does not inadvertently decode the same input twice (CWE-174). Such errors could be used to bypass allowlist schemes by introducing dangerous inputs after they have been checked. Use libraries such as the OWASP ESAPI Canonicalization control. Consider performing repeated canonicalization until your input does not change any more. This will avoid double-decoding and similar scenarios, but it might inadvertently modify inputs that are allowed to contain properly-encoded dangerous content.",

"When exchanging data between components, ensure that both components are using the same character encoding. Ensure that the proper encoding is applied at each interface. Explicitly set the encoding you are using whenever the protocol allows you to do so.",

"Refactor your program so that you do not have to dynamically generate code.",

"Run your code in a jail or similar sandbox environment that enforces strict boundaries between the process and the operating system. This may effectively restrict which code can be executed by your software. Examples include the Unix chroot jail and AppArmor. In general, managed code may provide some protection. This may not be a feasible solution, and it only limits the impact to the operating system; the rest of your application may still be subject to compromise. Be careful to avoid CWE-243 and other weaknesses related to jails.",

"Assume all input is malicious. Use an accept known good input validation strategy, i.e., use a list of acceptable inputs that strictly conform to specifications. Reject any input that does not strictly conform to specifications, or transform it into something that does. When performing input validation, consider all potentially relevant properties, including length, type of input, the full range of acceptable values, missing or extra inputs, syntax, consistency across related fields, and conformance to business rules. As an example of business rule logic, boat may be syntactically valid because it only contains alphanumeric characters, but it is not valid if the input is only expected to contain colors such as red or blue. Do not rely exclusively on looking for malicious or malformed inputs. This is likely to miss at least one undesirable input, especially if the code's environment changes. This can give attackers enough room to bypass the intended validation. However, denylists can be useful for detecting potential attacks or determining which inputs are so malformed that they should be rejected outright. To reduce the likelihood of code injection, use stringent allowlists that limit which constructs are allowed. If you are dynamically constructing code that invokes a function, then verifying that the input is alphanumeric might be insufficient. An attacker might still be able to reference a dangerous function that you did not intend to allow, such as system(), exec(), or exit().",

"Use automated static analysis tools that target this type of weakness. Many modern techniques use data flow analysis to minimize the number of false positives. This is not a perfect solution, since 100% accuracy and coverage are not feasible.",

"Use dynamic tools and techniques that interact with the software using large test suites with many diverse inputs, such as fuzz testing (fuzzing), robustness testing, and fault injection. The software's operation may slow down, but it should not become unstable, crash, or generate incorrect results.",

"Run the code in an environment that performs automatic taint propagation and prevents any command execution that uses tainted variables, such as Perl's -T switch. This will force the program to perform validation steps that remove the taint, although you must be careful to correctly validate your inputs so that you do not accidentally mark dangerous inputs as untainted (see CWE-183 and CWE-184).",

"Run the code in an environment that performs automatic taint propagation and prevents any command execution that uses tainted variables, such as Perl's -T switch. This will force the program to perform validation steps that remove the taint, although you must be careful to correctly validate your inputs so that you do not accidentally mark dangerous inputs as untainted (see CWE-183 and CWE-184).",

"Use a web server that employs a strict HTTP parsing procedure, such as Apache [REF-433].",
"Use only SSL communication.",
"Terminate the client session after each request.",
"Turn all pages to non-cacheable."    
]

print(len(a))
b = list(set(a))
print(len(b))

40
38


In [24]:
# Sumarize Mitigation
# summarize text
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

# def text_summarize(text, per):
#     nlp = spacy.load('en_core_web_sm')
#     doc= nlp(text)
#     tokens=[token.text for token in doc]
#     word_frequencies={}
#     for word in doc:
#         if word.text.lower() not in list(STOP_WORDS):
#             if word.text.lower() not in punctuation:
#                 if word.text not in word_frequencies.keys():
#                     word_frequencies[word.text] = 1
#                 else:
#                     word_frequencies[word.text] += 1
#     max_frequency=max(word_frequencies.values())
#     for word in word_frequencies.keys():
#         word_frequencies[word]=word_frequencies[word]/max_frequency
#     sentence_tokens= [sent for sent in doc.sents]
#     sentence_scores = {}
#     for sent in sentence_tokens:
#         for word in sent:
#             if word.text.lower() in word_frequencies.keys():
#                 if sent not in sentence_scores.keys():
#                     sentence_scores[sent]=word_frequencies[word.text.lower()]
#                 else:
#                     sentence_scores[sent]+=word_frequencies[word.text.lower()]
#     select_length=int(len(sentence_tokens)*per)
#     summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
#     final_summary=[word.text for word in summary]
#     summary=''.join(final_summary)
#     return summary

def text_summarize(text, per):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    sentences = [sent for sent in doc.sents]
    words = [word.text.lower() for word in doc if not word.is_stop and not word.is_punct]
    word_frequencies = {word: words.count(word) / len(words) for word in set(words)}
    sentence_scores = {sent: sum(word_frequencies.get(word.text.lower(), 0) for word in sent) for sent in sentences}
    summary = ''.join([sent.text for sent in nlargest(int(len(sentences) * per), sentence_scores, key=sentence_scores.get)])
    return summary


p_dets_short = []
p_dets = b
for i in range(len(p_dets)):
    x = text_summarize(p_dets[i], 0.25)
    if x != '':
        print()
        print('** Original text len', len(p_dets[i]), ':', p_dets[i])
        print('## Summary len', len(x), ':', x)
        p_dets_short.append(x)
    else:
        print()
        print('** Original short text len', len(p_dets[i]), ':', p_dets[i])
        p_dets_short.append(p_dets[i])


** Original short text len 193 : Use a CPU and operating system that offers Data Execution Protection (NX) or its equivalent [REF-60] [REF-61].Avoid making conversion between numeric types. Always check for the allowed ranges.

** Original short text len 88 : Use a web server that employs a strict HTTP parsing procedure, such as Apache [REF-433].

** Original short text len 262 : Use an input validation framework such as Struts or the OWASP ESAPI Validation API. Note that using a framework does not automatically address all input validation problems; be mindful of weaknesses that could arise from misusing the framework itself (CWE-1173).

** Original short text len 32 : Turn all pages to non-cacheable.

** Original short text len 398 : Run the code in an environment that performs automatic taint propagation and prevents any command execution that uses tainted variables, such as Perl's -T switch. This will force the program to perform validation steps that remove the taint, although yo


** Original short text len 27 : Use only SSL communication.

** Original short text len 217 : When the set of acceptable objects, such as filenames or URLs, is limited or known, create a mapping from a set of fixed input values (such as numeric IDs) to the actual filenames or URLs, and reject all other inputs.

** Original text len 600 : Consider adhering to the following rules when allocating and managing an application's memory: Double check that the buffer is as large as specified. When using functions that accept a number of bytes to copy, such as strncpy(), be aware that if the destination buffer size is equal to the source buffer size, it may not NULL-terminate the string. Check buffer boundaries if accessing the buffer in a loop and make sure there is no danger of writing past the allocated space. If necessary, truncate all input strings to a reasonable length before passing them to the copy and concatenation functions.
## Summary len 197 : When using functions that accept a nu


** Original short text len 246 : When your application combines data from multiple sources, perform the validation after the sources have been combined. The individual data elements may pass the validation step but violate the intended restrictions after they have been combined.

** Original short text len 323 : Use a vetted library or framework that does not allow this weakness to occur or provides constructs that make this weakness easier to avoid. Examples of libraries and frameworks that make it easier to generate properly encoded output include Microsoft's Anti-XSS library, the OWASP ESAPI Encoding module, and Apache Wicket.

** Original text len 518 : Use a language that does not allow this weakness to occur or provides constructs that make this weakness easier to avoid. For example, many languages that perform their own memory management, such as Java and Perl, are not subject to buffer overflows. Other languages, such as Ada and C#, typically provide overflow protection, but t

In [ ]:
Con_a
::SCOPE:Availability
        :IMPACT:DoS: Crash, Exit, or Restart
        :IMPACT:DoS: Resource Consumption (CPU)
        :IMPACT:DoS: Resource Consumption (Memory)
        :NOTE:An attacker could provide unexpected values and cause a program crash or excessive consumption of resources, such as memory and CPU.
::SCOPE:Confidentiality
        :IMPACT:Read Memory
        :IMPACT:Read Files or Directories
        :NOTE:An attacker could read confidential data if they are able to control resource references.
::SCOPE:Integrity
        :SCOPE:Confidentiality
        :SCOPE:Availability
            :IMPACT:Modify Memory
            :IMPACT:Execute Unauthorized Code or Commands
            :NOTE:An attacker could use malicious input to modify data or possibly alter control flow in unexpected ways, including arbitrary command execution.
::

Con_b
::SCOPE:Confidentiality
        :IMPACT:Read Application Data
        :NOTE:The confidentiality of sensitive data may be compromised by the use of a broken or risky cryptographic algorithm.
::SCOPE:Integrity
        :IMPACT:Modify Application Data
        :NOTE:The integrity of sensitive data may be compromised by the use of a broken or risky cryptographic algorithm.
::SCOPE:Accountability
        :SCOPE:Non-Repudiation
            :IMPACT:Hide Activities
            :NOTE:If the cryptographic algorithm is used to ensure the identity of the source of the data (such as digital signatures), then a broken algorithm will compromise this scheme and the source of the data cannot be proven.
::
        
Con_c
::SCOPE:Access Control
        :SCOPE:Confidentiality
            :IMPACT:Bypass Protection Mechanism
            :IMPACT:Read Application Data
            :NOTE:The most common attack performed with cross-site scripting involves the disclosure of information stored in user cookies. Typically, a malicious user will craft a client-side script, which -- when parsed by a web browser -- performs some activity (such as sending all site cookies to a given E-mail address). This script will be loaded and run by each user visiting the web site. Since the site requesting to run the script has access to the cookies in question, the malicious script does also.
::SCOPE:Integrity
        :SCOPE:Confidentiality
        :SCOPE:Availability
            :IMPACT:Execute Unauthorized Code or Commands
            :NOTE:In some circumstances it may be possible to run arbitrary code on a victim's computer when cross-site scripting is combined with other flaws.
::SCOPE:Confidentiality
        :SCOPE:Integrity
        :SCOPE:Availability
        :SCOPE:Access Control
            :IMPACT:Execute Unauthorized Code or Commands
            :IMPACT:Bypass Protection Mechanism
            :IMPACT:Read Application Data
            :NOTE:The consequence of an XSS attack is the same regardless of whether it is stored or reflected. The difference is in how the payload arrives at the server. XSS can cause a variety of problems for the end user that range in severity from an annoyance to complete account compromise. Some cross-site scripting vulnerabilities can be exploited to manipulate or steal cookies, create requests that can be mistaken for those of a valid user, compromise confidential information, or execute malicious code on the end user systems for a variety of nefarious purposes. Other damaging attacks include the disclosure of end user files, installation of Trojan horse programs, redirecting the user to some other page or site, running Active X controls (under Microsoft Internet Explorer) from sites that a user perceives as trustworthy, and modifying presentation of content.
::

In [ ]:
mit_b
::PHASE:Architecture and Design Operation
        :DESCRIPTION:Very carefully manage the setting, management, and handling of privileges. Explicitly manage trust zones in the software.
::PHASE:Architecture and Design
        :STRATEGY:Separation of Privilege
        :DESCRIPTION:Follow the principle of least privilege when assigning access rights to entities in a software system.
::PHASE:Architecture and Design
        :STRATEGY:Separation of Privilege
        :DESCRIPTION:Consider following the principle of separation of privilege. Require multiple conditions to be met before permitting access to a system resource.
::

mit_a
::PHASE:Architecture and Design
        :STRATEGY:Attack Surface Reduction
        :DESCRIPTION:Consider using language-theoretic security (LangSec) techniques that characterizes inputs using a formal language and builds recognizers for that language. This effectively requires parsing to be a distinct layer that effectively enforces a boundary between raw input and internal data representations, instead of allowing parser code to be scattered throughout the program, where it could be subject to errors or inconsistencies that create weaknesses. [REF-1109] [REF-1110] [REF-1111]
::PHASE:Architecture and Design
        :STRATEGY:Libraries or Frameworks
        :DESCRIPTION:Use an input validation framework such as Struts or the OWASP ESAPI Validation API. Note that using a framework does not automatically address all input validation problems; be mindful of weaknesses that could arise from misusing the framework itself (CWE-1173).
::PHASE:Architecture and Design
        :DESCRIPTION:For any security checks that are performed on the client side, ensure that these checks are duplicated on the server side, in order to avoid CWE-602. Attackers can bypass the client-side checks by modifying values after the checks have been performed, or by changing the client to remove the client-side checks entirely. Then, these modified values would be submitted to the server. Even though client-side checks provide minimal benefits with respect to server-side security, they are still useful. First, they can support intrusion detection. If the server receives input that should have been rejected by the client, then it may be an indication of an attack. Second, client-side error-checking can provide helpful feedback to the user about the expectations for valid input. Third, there may be a reduction in server-side processing time for accidental input errors, although this is typically a small savings.
::PHASE:Architecture and Design Implementation
        :STRATEGY:Attack Surface Reduction
        :DESCRIPTION:Understand all the potential areas where untrusted inputs can enter your software: parameters or arguments, cookies, anything read from the network, environment variables, reverse DNS lookups, query results, request headers, URL components, e-mail, files, filenames, databases, and any external systems that provide data to the application. Remember that such inputs may be obtained indirectly through API calls.
::PHASE:Implementation
        :STRATEGY:Input Validation
        :DESCRIPTION:Assume all input is malicious. Use an accept known good input validation strategy, i.e., use a list of acceptable inputs that strictly conform to specifications. Reject any input that does not strictly conform to specifications, or transform it into something that does. When performing input validation, consider all potentially relevant properties, including length, type of input, the full range of acceptable values, missing or extra inputs, syntax, consistency across related fields, and conformance to business rules. As an example of business rule logic, boat may be syntactically valid because it only contains alphanumeric characters, but it is not valid if the input is only expected to contain colors such as red or blue. Do not rely exclusively on looking for malicious or malformed inputs. This is likely to miss at least one undesirable input, especially if the code's environment changes. This can give attackers enough room to bypass the intended validation. However, denylists can be useful for detecting potential attacks or determining which inputs are so malformed that they should be rejected outright.
        :EFFECTIVENESS:High
::PHASE:Implementation
        :DESCRIPTION:When your application combines data from multiple sources, perform the validation after the sources have been combined. The individual data elements may pass the validation step but violate the intended restrictions after they have been combined.
::PHASE:Implementation
        :DESCRIPTION:Be especially careful to validate all input when invoking code that crosses language boundaries, such as from an interpreted language to native code. This could create an unexpected interaction between the language boundaries. Ensure that you are not violating any of the expectations of the language with which you are interfacing. For example, even though Java may not be susceptible to buffer overflows, providing a large argument in a call to native code might trigger an overflow.
::PHASE:Implementation
        :DESCRIPTION:Directly convert your input type into the expected data type, such as using a conversion function that translates a string into a number. After converting to the expected data type, ensure that the input's values fall within the expected range of allowable values and that multi-field consistencies are maintained.
::PHASE:Implementation
        :DESCRIPTION:Inputs should be decoded and canonicalized to the application's current internal representation before being validated (CWE-180, CWE-181). Make sure that your application does not inadvertently decode the same input twice (CWE-174). Such errors could be used to bypass allowlist schemes by introducing dangerous inputs after they have been checked. Use libraries such as the OWASP ESAPI Canonicalization control. Consider performing repeated canonicalization until your input does not change any more. This will avoid double-decoding and similar scenarios, but it might inadvertently modify inputs that are allowed to contain properly-encoded dangerous content.
::PHASE:Implementation
        :DESCRIPTION:When exchanging data between components, ensure that both components are using the same character encoding. Ensure that the proper encoding is applied at each interface. Explicitly set the encoding you are using whenever the protocol allows you to do so.
::

In [ ]:
test_dict = {'Gfg' : {"a" : 7, "b" : 9, "c" : 12}, 
             'is' : {"a" : 15, "b" : 19, "c" : 20},  
             'best' :{"a" : 5, "b" : 10, "c" : 2}} 
  
# printing original dictionary 
print("The original dictionary is : " + str(test_dict)) 
  
# initializing key 
temp = "c"
  
# using item() to extract key value pair as whole 
res = [val[temp] for key, val in test_dict.items() if temp in val] 
  
# printing result  
print("The extracted values : " + str(res))  

In [ ]:
a = {
  "Architecture and Design Operation": {
    "DESCRIPTION": [
      "Very carefully manage the setting, management, and handling of privileges. Explicitly manage trust zones in the software."
    ]
  },
  "Architecture and Design": {
    "STRATEGY": [
      "Separation of Privilege"
    ],
    "DESCRIPTION": [
      "Follow the principle of least privilege when assigning access rights to entities in a software system.",
      "Consider following the principle of separation of privilege. Require multiple conditions to be met before permitting access to a system resource."
    ]
  }
}

print(type(a))
xx = 'STRATEGY'
items = [val[xx] for key, val in a.items() if xx in val]
items = [item for sublist in items for item in sublist] # make one list, remove sublists
print('Strategy:', item_strategy)


In [ ]:
# Approach 1 - Using MITRE Latest Online API
from urllib.request import urlopen
import json

def get_cve_data(cve_id):
    cve_details_url = "https://cveawg.mitre.org/api/cve/"+cve_id
    print(' *', cve_id, '*', cve_details_url, '*')

    full_data, short_data = {}, {}
    try:
        response = urlopen(cve_details_url)
        full_data = json.loads(response.read())
        
        # get product version(s)
        prod_vers, prob_types = [], []
        for v in range(0, len(full_data['containers']['cna']['affected'][0]['versions'])):
            #print(full_data['containers']['cna']['affected'][0]['versions'][v]['version'])
            prod_vers.append(full_data['containers']['cna']['affected'][0]['versions'][v]['version']) 
        
        # get problem type(s) related to vulnerability
        # print(len(full_data['containers']['cna']['problemTypes']))
        for p in range(0, len(full_data['containers']['cna']['problemTypes'])):
            # print(full_data['containers']['cna']['problemTypes'][p]['descriptions'])    
            for pd in range(0, len(full_data['containers']['cna']['problemTypes'][p]['descriptions'])):
                # print(full_data['containers']['cna']['problemTypes'][p]['descriptions'][pd]['description'])
                prob_types.append(full_data['containers']['cna']['problemTypes'][p]['descriptions'][pd]['description'])
        
        short_data = {
            "cve_id": cve_id,
            "description": full_data['containers']['cna']['descriptions'][0]['value'],
            "vendor": full_data['containers']['cna']['affected'][0]['vendor'],
            "affected_product": full_data['containers']['cna']['affected'][0]['product'],
            "version(s)": prod_vers, 
            "problem_type(s)": prob_types
        }
        
        # Clear 'n/a' fields!
        if short_data['vendor'] == 'n/a': del short_data['vendor']
        if short_data['affected_product'] == 'n/a': del short_data['affected_product']
        if short_data['version(s)'] == ['n/a']: del short_data['version(s)']
        if short_data['problem_type(s)'] == ['n/a']: del short_data['problem_type(s)']
        
    except:
        full_data[cve_id], short_data[cve_id] = "CVE not found", "CVE not found"
    
    print('Description from MITRE:', short_data['description'])
    print(json.dumps(short_data, indent=2))
    
    # return full_data, short_data

cve_id = 'CVE-2013-1752'# 'CVE-2022-46179'# 'CVE-2019-0027'# CVE-2022-0074 CVE-2022-0030 CVE-2021-0065 CVE-2021-0007 CVE-2020-0031 CVE-2019-0027
get_cve_data(cve_id)

In [ ]:
lookupitems = ['CVE-1999', 'CVE-1999-0003', 'CVE-2020-0002', 'CVE-1999-0001', 'CVE-1999-1136', 'CVE-2000-0030', 'CVE-2010-0001', 'CVE-2015-0001','CVE-2015-0030']
for cve_id in lookupitems:
    get_cve_data(cve_id)
    print('---------------------------------------------------------\n')

In [ ]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'A': 'foo bar foo bar foo bar foo foo'.split(),
                   'B': 'one one two three two two one three'.split(),
                   'C': np.arange(8), 'D': np.arange(8) * 2})
print(df)
print()
print(df.loc[df['A'] == 'foo'])
print()
print(df.loc[df['C'] == 1])
print()
print(df.loc[df['C'] == 1]['A'][1])

In [ ]:
print(df.loc[df['A'] == 'foo'].iloc[:1])
print()
print(df.loc[df['A'] == 'foo'].iloc[:1]['B'][0])